In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 1. Деревья Решений

1) Подключаем набор данных

In [ ]:
df = pd.read_csv('/kaggle/input/adult-income-dataset/adult.csv')

In [ ]:
df['race'].value_counts()

In [ ]:
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})
df['gender'] = df['gender'].map({'Male': 0, 'Female':1})
df['race'] = df['race'].map({'White': 0, 'Black': 1, 'Asian-Pac-Islander': 2, 'Amer-Indian-Eskimo': 3,
                            'Other': 4})

df.head()

2) Разбиваем набор данных на обучающую и тестовую выборки

In [ ]:
X = df[['age', 'educational-num', 'race', 'gender', 'hours-per-week']]
y = df['income']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state = 8)

In [ ]:
y.value_counts()

In [ ]:
list(X.keys())

3) Обучаем алгоритм классификации и проверяем качество модели

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 3, random_state = 8)
tree.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = tree.predict(X_valid)
print(accuracy_score(y_valid, y_pred),
f1_score(y_valid, y_pred, average = 'micro'))

# 2. Настройка гиперпараметров дерева

1) Создаем генератор разбиений

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5 , shuffle = True, random_state = 42)

2) Осуществим кросс-валидацию модели для подбора гиперпараметров

In [ ]:
from sklearn.model_selection import GridSearchCV
tree_params = {'max_depth': np.arange(2, 11), 'min_samples_split': np.arange(2, 10), 
               'min_samples_leaf': np.arange(1,10), 'max_features': np.arange(1, 5)}

tree_grid = GridSearchCV(tree, tree_params, cv=5, scoring='accuracy')
tree_grid.fit(X_train, y_train)

In [ ]:
tree_grid.best_params_

In [ ]:
tree_grid.best_score_

3) Построим полученное дерево 

[](http://dreampuf.github.io/GraphvizOnline/#digraph%20Tree%20%7B%0Anode%20%5Bshape%3Dbox%5D%20%3B%0A0%20%5Blabel%3D%22educational-num%20%3C%3D%2012.5%5Cngini%20%3D%200.365%5Cnsamples%20%3D%2034189%5Cnvalue%20%3D%20%5B25991%2C%208198%5D%22%5D%20%3B%0A1%20%5Blabel%3D%22age%20%3C%3D%2030.5%5Cngini%20%3D%200.27%5Cnsamples%20%3D%2025713%5Cnvalue%20%3D%20%5B21578%2C%204135%5D%22%5D%20%3B%0A0%20-%3E%201%20%5Blabeldistance%3D2.5%2C%20labelangle%3D45%2C%20headlabel%3D%22True%22%5D%20%3B%0A2%20%5Blabel%3D%22age%20%3C%3D%2024.5%5Cngini%20%3D%200.079%5Cnsamples%20%3D%209001%5Cnvalue%20%3D%20%5B8632%2C%20369%5D%22%5D%20%3B%0A1%20-%3E%202%20%3B%0A3%20%5Blabel%3D%22gini%20%3D%200.017%5Cnsamples%20%3D%205251%5Cnvalue%20%3D%20%5B5207%2C%2044%5D%22%5D%20%3B%0A2%20-%3E%203%20%3B%0A4%20%5Blabel%3D%22gini%20%3D%200.158%5Cnsamples%20%3D%203750%5Cnvalue%20%3D%20%5B3425%2C%20325%5D%22%5D%20%3B%0A2%20-%3E%204%20%3B%0A5%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.349%5Cnsamples%20%3D%2016712%5Cnvalue%20%3D%20%5B12946%2C%203766%5D%22%5D%20%3B%0A1%20-%3E%205%20%3B%0A6%20%5Blabel%3D%22gini%20%3D%200.404%5Cnsamples%20%3D%2011587%5Cnvalue%20%3D%20%5B8330%2C%203257%5D%22%5D%20%3B%0A5%20-%3E%206%20%3B%0A7%20%5Blabel%3D%22gini%20%3D%200.179%5Cnsamples%20%3D%205125%5Cnvalue%20%3D%20%5B4616%2C%20509%5D%22%5D%20%3B%0A5%20-%3E%207%20%3B%0A8%20%5Blabel%3D%22age%20%3C%3D%2028.5%5Cngini%20%3D%200.499%5Cnsamples%20%3D%208476%5Cnvalue%20%3D%20%5B4413%2C%204063%5D%22%5D%20%3B%0A0%20-%3E%208%20%5Blabeldistance%3D2.5%2C%20labelangle%3D-45%2C%20headlabel%3D%22False%22%5D%20%3B%0A9%20%5Blabel%3D%22age%20%3C%3D%2027.5%5Cngini%20%3D%200.197%5Cnsamples%20%3D%201473%5Cnvalue%20%3D%20%5B1310%2C%20163%5D%22%5D%20%3B%0A8%20-%3E%209%20%3B%0A10%20%5Blabel%3D%22gini%20%3D%200.143%5Cnsamples%20%3D%201242%5Cnvalue%20%3D%20%5B1146%2C%2096%5D%22%5D%20%3B%0A9%20-%3E%2010%20%3B%0A11%20%5Blabel%3D%22gini%20%3D%200.412%5Cnsamples%20%3D%20231%5Cnvalue%20%3D%20%5B164%2C%2067%5D%22%5D%20%3B%0A9%20-%3E%2011%20%3B%0A12%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.494%5Cnsamples%20%3D%207003%5Cnvalue%20%3D%20%5B3103%2C%203900%5D%22%5D%20%3B%0A8%20-%3E%2012%20%3B%0A13%20%5Blabel%3D%22gini%20%3D%200.465%5Cnsamples%20%3D%205221%5Cnvalue%20%3D%20%5B1916%2C%203305%5D%22%5D%20%3B%0A12%20-%3E%2013%20%3B%0A14%20%5Blabel%3D%22gini%20%3D%200.445%5Cnsamples%20%3D%201782%5Cnvalue%20%3D%20%5B1187%2C%20595%5D%22%5D%20%3B%0A12%20-%3E%2014%20%3B%0A%7D)

In [ ]:
optimal_tree = DecisionTreeClassifier(max_depth = 7, max_features = 4, min_samples_leaf = 9, 
                                      min_samples_split = 2, random_state = 8)
optimal_tree.fit(X_train, y_train)
y_pred = optimal_tree.predict(X_valid)
print(accuracy_score(y_valid, y_pred))

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(optimal_tree, out_file='tree.dot', feature_names=X.columns)
#print(open('tree.dot').read()) 

[](https://dreampuf.github.io/GraphvizOnline/#digraph%20Tree%20%7B%0Anode%20%5Bshape%3Dbox%5D%20%3B%0A0%20%5Blabel%3D%22educational-num%20%3C%3D%2012.5%5Cngini%20%3D%200.365%5Cnsamples%20%3D%2034189%5Cnvalue%20%3D%20%5B25991%2C%208198%5D%22%5D%20%3B%0A1%20%5Blabel%3D%22age%20%3C%3D%2030.5%5Cngini%20%3D%200.27%5Cnsamples%20%3D%2025713%5Cnvalue%20%3D%20%5B21578%2C%204135%5D%22%5D%20%3B%0A0%20-%3E%201%20%5Blabeldistance%3D2.5%2C%20labelangle%3D45%2C%20headlabel%3D%22True%22%5D%20%3B%0A2%20%5Blabel%3D%22hours-per-week%20%3C%3D%2041.5%5Cngini%20%3D%200.079%5Cnsamples%20%3D%209001%5Cnvalue%20%3D%20%5B8632%2C%20369%5D%22%5D%20%3B%0A1%20-%3E%202%20%3B%0A3%20%5Blabel%3D%22hours-per-week%20%3C%3D%2035.5%5Cngini%20%3D%200.057%5Cnsamples%20%3D%207453%5Cnvalue%20%3D%20%5B7235%2C%20218%5D%22%5D%20%3B%0A2%20-%3E%203%20%3B%0A4%20%5Blabel%3D%22educational-num%20%3C%3D%2010.5%5Cngini%20%3D%200.015%5Cnsamples%20%3D%203214%5Cnvalue%20%3D%20%5B3189%2C%2025%5D%22%5D%20%3B%0A3%20-%3E%204%20%3B%0A5%20%5Blabel%3D%22age%20%3C%3D%2027.5%5Cngini%20%3D%200.012%5Cnsamples%20%3D%203049%5Cnvalue%20%3D%20%5B3030%2C%2019%5D%22%5D%20%3B%0A4%20-%3E%205%20%3B%0A6%20%5Blabel%3D%22age%20%3C%3D%2021.5%5Cngini%20%3D%200.005%5Cnsamples%20%3D%202800%5Cnvalue%20%3D%20%5B2793%2C%207%5D%22%5D%20%3B%0A5%20-%3E%206%20%3B%0A7%20%5Blabel%3D%22gini%20%3D%200.001%5Cnsamples%20%3D%201962%5Cnvalue%20%3D%20%5B1961%2C%201%5D%22%5D%20%3B%0A6%20-%3E%207%20%3B%0A8%20%5Blabel%3D%22gini%20%3D%200.014%5Cnsamples%20%3D%20838%5Cnvalue%20%3D%20%5B832%2C%206%5D%22%5D%20%3B%0A6%20-%3E%208%20%3B%0A9%20%5Blabel%3D%22hours-per-week%20%3C%3D%2015.5%5Cngini%20%3D%200.092%5Cnsamples%20%3D%20249%5Cnvalue%20%3D%20%5B237%2C%2012%5D%22%5D%20%3B%0A5%20-%3E%209%20%3B%0A10%20%5Blabel%3D%22gini%20%3D%200.198%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B24%2C%203%5D%22%5D%20%3B%0A9%20-%3E%2010%20%3B%0A11%20%5Blabel%3D%22gini%20%3D%200.078%5Cnsamples%20%3D%20222%5Cnvalue%20%3D%20%5B213%2C%209%5D%22%5D%20%3B%0A9%20-%3E%2011%20%3B%0A12%20%5Blabel%3D%22age%20%3C%3D%2028.5%5Cngini%20%3D%200.07%5Cnsamples%20%3D%20165%5Cnvalue%20%3D%20%5B159%2C%206%5D%22%5D%20%3B%0A4%20-%3E%2012%20%3B%0A13%20%5Blabel%3D%22age%20%3C%3D%2023.5%5Cngini%20%3D%200.04%5Cnsamples%20%3D%20148%5Cnvalue%20%3D%20%5B145%2C%203%5D%22%5D%20%3B%0A12%20-%3E%2013%20%3B%0A14%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2083%5Cnvalue%20%3D%20%5B83%2C%200%5D%22%5D%20%3B%0A13%20-%3E%2014%20%3B%0A15%20%5Blabel%3D%22gini%20%3D%200.088%5Cnsamples%20%3D%2065%5Cnvalue%20%3D%20%5B62%2C%203%5D%22%5D%20%3B%0A13%20-%3E%2015%20%3B%0A16%20%5Blabel%3D%22educational-num%20%3C%3D%2011.5%5Cngini%20%3D%200.291%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B14%2C%203%5D%22%5D%20%3B%0A12%20-%3E%2016%20%3B%0A17%20%5Blabel%3D%22gini%20%3D%200.346%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B7%2C%202%5D%22%5D%20%3B%0A16%20-%3E%2017%20%3B%0A18%20%5Blabel%3D%22gini%20%3D%200.219%5Cnsamples%20%3D%208%5Cnvalue%20%3D%20%5B7%2C%201%5D%22%5D%20%3B%0A16%20-%3E%2018%20%3B%0A19%20%5Blabel%3D%22age%20%3C%3D%2026.5%5Cngini%20%3D%200.087%5Cnsamples%20%3D%204239%5Cnvalue%20%3D%20%5B4046%2C%20193%5D%22%5D%20%3B%0A3%20-%3E%2019%20%3B%0A20%20%5Blabel%3D%22age%20%3C%3D%2024.5%5Cngini%20%3D%200.039%5Cnsamples%20%3D%202727%5Cnvalue%20%3D%20%5B2673%2C%2054%5D%22%5D%20%3B%0A19%20-%3E%2020%20%3B%0A21%20%5Blabel%3D%22age%20%3C%3D%2022.5%5Cngini%20%3D%200.02%5Cnsamples%20%3D%202027%5Cnvalue%20%3D%20%5B2007%2C%2020%5D%22%5D%20%3B%0A20%20-%3E%2021%20%3B%0A22%20%5Blabel%3D%22gini%20%3D%200.009%5Cnsamples%20%3D%201330%5Cnvalue%20%3D%20%5B1324%2C%206%5D%22%5D%20%3B%0A21%20-%3E%2022%20%3B%0A23%20%5Blabel%3D%22gini%20%3D%200.039%5Cnsamples%20%3D%20697%5Cnvalue%20%3D%20%5B683%2C%2014%5D%22%5D%20%3B%0A21%20-%3E%2023%20%3B%0A24%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.092%5Cnsamples%20%3D%20700%5Cnvalue%20%3D%20%5B666%2C%2034%5D%22%5D%20%3B%0A20%20-%3E%2024%20%3B%0A25%20%5Blabel%3D%22gini%20%3D%200.048%5Cnsamples%20%3D%20448%5Cnvalue%20%3D%20%5B437%2C%2011%5D%22%5D%20%3B%0A24%20-%3E%2025%20%3B%0A26%20%5Blabel%3D%22gini%20%3D%200.166%5Cnsamples%20%3D%20252%5Cnvalue%20%3D%20%5B229%2C%2023%5D%22%5D%20%3B%0A24%20-%3E%2026%20%3B%0A27%20%5Blabel%3D%22educational-num%20%3C%3D%207.5%5Cngini%20%3D%200.167%5Cnsamples%20%3D%201512%5Cnvalue%20%3D%20%5B1373%2C%20139%5D%22%5D%20%3B%0A19%20-%3E%2027%20%3B%0A28%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.04%5Cnsamples%20%3D%20196%5Cnvalue%20%3D%20%5B192%2C%204%5D%22%5D%20%3B%0A27%20-%3E%2028%20%3B%0A29%20%5Blabel%3D%22gini%20%3D%200.054%5Cnsamples%20%3D%20143%5Cnvalue%20%3D%20%5B139%2C%204%5D%22%5D%20%3B%0A28%20-%3E%2029%20%3B%0A30%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2053%5Cnvalue%20%3D%20%5B53%2C%200%5D%22%5D%20%3B%0A28%20-%3E%2030%20%3B%0A31%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.184%5Cnsamples%20%3D%201316%5Cnvalue%20%3D%20%5B1181%2C%20135%5D%22%5D%20%3B%0A27%20-%3E%2031%20%3B%0A32%20%5Blabel%3D%22gini%20%3D%200.218%5Cnsamples%20%3D%20850%5Cnvalue%20%3D%20%5B744%2C%20106%5D%22%5D%20%3B%0A31%20-%3E%2032%20%3B%0A33%20%5Blabel%3D%22gini%20%3D%200.117%5Cnsamples%20%3D%20466%5Cnvalue%20%3D%20%5B437%2C%2029%5D%22%5D%20%3B%0A31%20-%3E%2033%20%3B%0A34%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.176%5Cnsamples%20%3D%201548%5Cnvalue%20%3D%20%5B1397%2C%20151%5D%22%5D%20%3B%0A2%20-%3E%2034%20%3B%0A35%20%5Blabel%3D%22age%20%3C%3D%2023.5%5Cngini%20%3D%200.195%5Cnsamples%20%3D%201251%5Cnvalue%20%3D%20%5B1114%2C%20137%5D%22%5D%20%3B%0A34%20-%3E%2035%20%3B%0A36%20%5Blabel%3D%22age%20%3C%3D%2021.5%5Cngini%20%3D%200.039%5Cnsamples%20%3D%20348%5Cnvalue%20%3D%20%5B341%2C%207%5D%22%5D%20%3B%0A35%20-%3E%2036%20%3B%0A37%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20175%5Cnvalue%20%3D%20%5B175%2C%200%5D%22%5D%20%3B%0A36%20-%3E%2037%20%3B%0A38%20%5Blabel%3D%22educational-num%20%3C%3D%208.5%5Cngini%20%3D%200.078%5Cnsamples%20%3D%20173%5Cnvalue%20%3D%20%5B166%2C%207%5D%22%5D%20%3B%0A36%20-%3E%2038%20%3B%0A39%20%5Blabel%3D%22gini%20%3D%200.236%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B19%2C%203%5D%22%5D%20%3B%0A38%20-%3E%2039%20%3B%0A40%20%5Blabel%3D%22gini%20%3D%200.052%5Cnsamples%20%3D%20151%5Cnvalue%20%3D%20%5B147%2C%204%5D%22%5D%20%3B%0A38%20-%3E%2040%20%3B%0A41%20%5Blabel%3D%22hours-per-week%20%3C%3D%2064.0%5Cngini%20%3D%200.246%5Cnsamples%20%3D%20903%5Cnvalue%20%3D%20%5B773%2C%20130%5D%22%5D%20%3B%0A35%20-%3E%2041%20%3B%0A42%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.232%5Cnsamples%20%3D%20790%5Cnvalue%20%3D%20%5B684%2C%20106%5D%22%5D%20%3B%0A41%20-%3E%2042%20%3B%0A43%20%5Blabel%3D%22gini%20%3D%200.193%5Cnsamples%20%3D%20452%5Cnvalue%20%3D%20%5B403%2C%2049%5D%22%5D%20%3B%0A42%20-%3E%2043%20%3B%0A44%20%5Blabel%3D%22gini%20%3D%200.28%5Cnsamples%20%3D%20338%5Cnvalue%20%3D%20%5B281%2C%2057%5D%22%5D%20%3B%0A42%20-%3E%2044%20%3B%0A45%20%5Blabel%3D%22hours-per-week%20%3C%3D%2073.5%5Cngini%20%3D%200.335%5Cnsamples%20%3D%20113%5Cnvalue%20%3D%20%5B89%2C%2024%5D%22%5D%20%3B%0A41%20-%3E%2045%20%3B%0A46%20%5Blabel%3D%22gini%20%3D%200.412%5Cnsamples%20%3D%2062%5Cnvalue%20%3D%20%5B44%2C%2018%5D%22%5D%20%3B%0A45%20-%3E%2046%20%3B%0A47%20%5Blabel%3D%22gini%20%3D%200.208%5Cnsamples%20%3D%2051%5Cnvalue%20%3D%20%5B45%2C%206%5D%22%5D%20%3B%0A45%20-%3E%2047%20%3B%0A48%20%5Blabel%3D%22age%20%3C%3D%2024.5%5Cngini%20%3D%200.09%5Cnsamples%20%3D%20297%5Cnvalue%20%3D%20%5B283%2C%2014%5D%22%5D%20%3B%0A34%20-%3E%2048%20%3B%0A49%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.033%5Cnsamples%20%3D%20121%5Cnvalue%20%3D%20%5B119%2C%202%5D%22%5D%20%3B%0A48%20-%3E%2049%20%3B%0A50%20%5Blabel%3D%22hours-per-week%20%3C%3D%2043.5%5Cngini%20%3D%200.059%5Cnsamples%20%3D%2066%5Cnvalue%20%3D%20%5B64%2C%202%5D%22%5D%20%3B%0A49%20-%3E%2050%20%3B%0A51%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%205%5Cnvalue%20%3D%20%5B4%2C%201%5D%22%5D%20%3B%0A50%20-%3E%2051%20%3B%0A52%20%5Blabel%3D%22gini%20%3D%200.032%5Cnsamples%20%3D%2061%5Cnvalue%20%3D%20%5B60%2C%201%5D%22%5D%20%3B%0A50%20-%3E%2052%20%3B%0A53%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2055%5Cnvalue%20%3D%20%5B55%2C%200%5D%22%5D%20%3B%0A49%20-%3E%2053%20%3B%0A54%20%5Blabel%3D%22educational-num%20%3C%3D%2011.5%5Cngini%20%3D%200.127%5Cnsamples%20%3D%20176%5Cnvalue%20%3D%20%5B164%2C%2012%5D%22%5D%20%3B%0A48%20-%3E%2054%20%3B%0A55%20%5Blabel%3D%22age%20%3C%3D%2025.5%5Cngini%20%3D%200.107%5Cnsamples%20%3D%20159%5Cnvalue%20%3D%20%5B150%2C%209%5D%22%5D%20%3B%0A54%20-%3E%2055%20%3B%0A56%20%5Blabel%3D%22gini%20%3D%200.185%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B26%2C%203%5D%22%5D%20%3B%0A55%20-%3E%2056%20%3B%0A57%20%5Blabel%3D%22gini%20%3D%200.088%5Cnsamples%20%3D%20130%5Cnvalue%20%3D%20%5B124%2C%206%5D%22%5D%20%3B%0A55%20-%3E%2057%20%3B%0A58%20%5Blabel%3D%22hours-per-week%20%3C%3D%2051.5%5Cngini%20%3D%200.291%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B14%2C%203%5D%22%5D%20%3B%0A54%20-%3E%2058%20%3B%0A59%20%5Blabel%3D%22gini%20%3D%200.42%5Cnsamples%20%3D%2010%5Cnvalue%20%3D%20%5B7%2C%203%5D%22%5D%20%3B%0A58%20-%3E%2059%20%3B%0A60%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%207%5Cnvalue%20%3D%20%5B7%2C%200%5D%22%5D%20%3B%0A58%20-%3E%2060%20%3B%0A61%20%5Blabel%3D%22hours-per-week%20%3C%3D%2040.5%5Cngini%20%3D%200.349%5Cnsamples%20%3D%2016712%5Cnvalue%20%3D%20%5B12946%2C%203766%5D%22%5D%20%3B%0A1%20-%3E%2061%20%3B%0A62%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.295%5Cnsamples%20%3D%2011838%5Cnvalue%20%3D%20%5B9710%2C%202128%5D%22%5D%20%3B%0A61%20-%3E%2062%20%3B%0A63%20%5Blabel%3D%22educational-num%20%3C%3D%208.5%5Cngini%20%3D%200.355%5Cnsamples%20%3D%207494%5Cnvalue%20%3D%20%5B5766%2C%201728%5D%22%5D%20%3B%0A62%20-%3E%2063%20%3B%0A64%20%5Blabel%3D%22age%20%3C%3D%2037.5%5Cngini%20%3D%200.137%5Cnsamples%20%3D%201498%5Cnvalue%20%3D%20%5B1387%2C%20111%5D%22%5D%20%3B%0A63%20-%3E%2064%20%3B%0A65%20%5Blabel%3D%22hours-per-week%20%3C%3D%2039.0%5Cngini%20%3D%200.051%5Cnsamples%20%3D%20341%5Cnvalue%20%3D%20%5B332%2C%209%5D%22%5D%20%3B%0A64%20-%3E%2065%20%3B%0A66%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2064%5Cnvalue%20%3D%20%5B64%2C%200%5D%22%5D%20%3B%0A65%20-%3E%2066%20%3B%0A67%20%5Blabel%3D%22gini%20%3D%200.063%5Cnsamples%20%3D%20277%5Cnvalue%20%3D%20%5B268%2C%209%5D%22%5D%20%3B%0A65%20-%3E%2067%20%3B%0A68%20%5Blabel%3D%22educational-num%20%3C%3D%205.5%5Cngini%20%3D%200.161%5Cnsamples%20%3D%201157%5Cnvalue%20%3D%20%5B1055%2C%20102%5D%22%5D%20%3B%0A64%20-%3E%2068%20%3B%0A69%20%5Blabel%3D%22gini%20%3D%200.11%5Cnsamples%20%3D%20649%5Cnvalue%20%3D%20%5B611%2C%2038%5D%22%5D%20%3B%0A68%20-%3E%2069%20%3B%0A70%20%5Blabel%3D%22gini%20%3D%200.22%5Cnsamples%20%3D%20508%5Cnvalue%20%3D%20%5B444%2C%2064%5D%22%5D%20%3B%0A68%20-%3E%2070%20%3B%0A71%20%5Blabel%3D%22age%20%3C%3D%2035.5%5Cngini%20%3D%200.394%5Cnsamples%20%3D%205996%5Cnvalue%20%3D%20%5B4379%2C%201617%5D%22%5D%20%3B%0A63%20-%3E%2071%20%3B%0A72%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.254%5Cnsamples%20%3D%201242%5Cnvalue%20%3D%20%5B1057%2C%20185%5D%22%5D%20%3B%0A71%20-%3E%2072%20%3B%0A73%20%5Blabel%3D%22gini%20%3D%200.202%5Cnsamples%20%3D%20721%5Cnvalue%20%3D%20%5B639%2C%2082%5D%22%5D%20%3B%0A72%20-%3E%2073%20%3B%0A74%20%5Blabel%3D%22gini%20%3D%200.317%5Cnsamples%20%3D%20521%5Cnvalue%20%3D%20%5B418%2C%20103%5D%22%5D%20%3B%0A72%20-%3E%2074%20%3B%0A75%20%5Blabel%3D%22hours-per-week%20%3C%3D%2035.5%5Cngini%20%3D%200.421%5Cnsamples%20%3D%204754%5Cnvalue%20%3D%20%5B3322%2C%201432%5D%22%5D%20%3B%0A71%20-%3E%2075%20%3B%0A76%20%5Blabel%3D%22gini%20%3D%200.228%5Cnsamples%20%3D%20837%5Cnvalue%20%3D%20%5B727%2C%20110%5D%22%5D%20%3B%0A75%20-%3E%2076%20%3B%0A77%20%5Blabel%3D%22gini%20%3D%200.447%5Cnsamples%20%3D%203917%5Cnvalue%20%3D%20%5B2595%2C%201322%5D%22%5D%20%3B%0A75%20-%3E%2077%20%3B%0A78%20%5Blabel%3D%22educational-num%20%3C%3D%208.5%5Cngini%20%3D%200.167%5Cnsamples%20%3D%204344%5Cnvalue%20%3D%20%5B3944%2C%20400%5D%22%5D%20%3B%0A62%20-%3E%2078%20%3B%0A79%20%5Blabel%3D%22age%20%3C%3D%2039.5%5Cngini%20%3D%200.049%5Cnsamples%20%3D%20682%5Cnvalue%20%3D%20%5B665%2C%2017%5D%22%5D%20%3B%0A78%20-%3E%2079%20%3B%0A80%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20182%5Cnvalue%20%3D%20%5B182%2C%200%5D%22%5D%20%3B%0A79%20-%3E%2080%20%3B%0A81%20%5Blabel%3D%22hours-per-week%20%3C%3D%2029.5%5Cngini%20%3D%200.066%5Cnsamples%20%3D%20500%5Cnvalue%20%3D%20%5B483%2C%2017%5D%22%5D%20%3B%0A79%20-%3E%2081%20%3B%0A82%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20129%5Cnvalue%20%3D%20%5B129%2C%200%5D%22%5D%20%3B%0A81%20-%3E%2082%20%3B%0A83%20%5Blabel%3D%22gini%20%3D%200.087%5Cnsamples%20%3D%20371%5Cnvalue%20%3D%20%5B354%2C%2017%5D%22%5D%20%3B%0A81%20-%3E%2083%20%3B%0A84%20%5Blabel%3D%22age%20%3C%3D%2060.5%5Cngini%20%3D%200.187%5Cnsamples%20%3D%203662%5Cnvalue%20%3D%20%5B3279%2C%20383%5D%22%5D%20%3B%0A78%20-%3E%2084%20%3B%0A85%20%5Blabel%3D%22hours-per-week%20%3C%3D%2024.5%5Cngini%20%3D%200.199%5Cnsamples%20%3D%203192%5Cnvalue%20%3D%20%5B2834%2C%20358%5D%22%5D%20%3B%0A84%20-%3E%2085%20%3B%0A86%20%5Blabel%3D%22gini%20%3D%200.271%5Cnsamples%20%3D%20315%5Cnvalue%20%3D%20%5B264%2C%2051%5D%22%5D%20%3B%0A85%20-%3E%2086%20%3B%0A87%20%5Blabel%3D%22gini%20%3D%200.191%5Cnsamples%20%3D%202877%5Cnvalue%20%3D%20%5B2570%2C%20307%5D%22%5D%20%3B%0A85%20-%3E%2087%20%3B%0A88%20%5Blabel%3D%22hours-per-week%20%3C%3D%2033.5%5Cngini%20%3D%200.101%5Cnsamples%20%3D%20470%5Cnvalue%20%3D%20%5B445%2C%2025%5D%22%5D%20%3B%0A84%20-%3E%2088%20%3B%0A89%20%5Blabel%3D%22gini%20%3D%200.073%5Cnsamples%20%3D%20262%5Cnvalue%20%3D%20%5B252%2C%2010%5D%22%5D%20%3B%0A88%20-%3E%2089%20%3B%0A90%20%5Blabel%3D%22gini%20%3D%200.134%5Cnsamples%20%3D%20208%5Cnvalue%20%3D%20%5B193%2C%2015%5D%22%5D%20%3B%0A88%20-%3E%2090%20%3B%0A91%20%5Blabel%3D%22educational-num%20%3C%3D%207.5%5Cngini%20%3D%200.446%5Cnsamples%20%3D%204874%5Cnvalue%20%3D%20%5B3236%2C%201638%5D%22%5D%20%3B%0A61%20-%3E%2091%20%3B%0A92%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.276%5Cnsamples%20%3D%20575%5Cnvalue%20%3D%20%5B480%2C%2095%5D%22%5D%20%3B%0A91%20-%3E%2092%20%3B%0A93%20%5Blabel%3D%22hours-per-week%20%3C%3D%2044.5%5Cngini%20%3D%200.301%5Cnsamples%20%3D%20488%5Cnvalue%20%3D%20%5B398%2C%2090%5D%22%5D%20%3B%0A92%20-%3E%2093%20%3B%0A94%20%5Blabel%3D%22age%20%3C%3D%2048.5%5Cngini%20%3D%200.452%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B19%2C%2010%5D%22%5D%20%3B%0A93%20-%3E%2094%20%3B%0A95%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B12%2C%204%5D%22%5D%20%3B%0A94%20-%3E%2095%20%3B%0A96%20%5Blabel%3D%22gini%20%3D%200.497%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B7%2C%206%5D%22%5D%20%3B%0A94%20-%3E%2096%20%3B%0A97%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.5%5Cngini%20%3D%200.288%5Cnsamples%20%3D%20459%5Cnvalue%20%3D%20%5B379%2C%2080%5D%22%5D%20%3B%0A93%20-%3E%2097%20%3B%0A98%20%5Blabel%3D%22gini%20%3D%200.196%5Cnsamples%20%3D%20136%5Cnvalue%20%3D%20%5B121%2C%2015%5D%22%5D%20%3B%0A97%20-%3E%2098%20%3B%0A99%20%5Blabel%3D%22gini%20%3D%200.321%5Cnsamples%20%3D%20323%5Cnvalue%20%3D%20%5B258%2C%2065%5D%22%5D%20%3B%0A97%20-%3E%2099%20%3B%0A100%20%5Blabel%3D%22age%20%3C%3D%2053.5%5Cngini%20%3D%200.108%5Cnsamples%20%3D%2087%5Cnvalue%20%3D%20%5B82%2C%205%5D%22%5D%20%3B%0A92%20-%3E%20100%20%3B%0A101%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2057%5Cnvalue%20%3D%20%5B57%2C%200%5D%22%5D%20%3B%0A100%20-%3E%20101%20%3B%0A102%20%5Blabel%3D%22educational-num%20%3C%3D%206.5%5Cngini%20%3D%200.278%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B25%2C%205%5D%22%5D%20%3B%0A100%20-%3E%20102%20%3B%0A103%20%5Blabel%3D%22gini%20%3D%200.159%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B21%2C%202%5D%22%5D%20%3B%0A102%20-%3E%20103%20%3B%0A104%20%5Blabel%3D%22gini%20%3D%200.49%5Cnsamples%20%3D%207%5Cnvalue%20%3D%20%5B4%2C%203%5D%22%5D%20%3B%0A102%20-%3E%20104%20%3B%0A105%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.46%5Cnsamples%20%3D%204299%5Cnvalue%20%3D%20%5B2756%2C%201543%5D%22%5D%20%3B%0A91%20-%3E%20105%20%3B%0A106%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.48%5Cnsamples%20%3D%203605%5Cnvalue%20%3D%20%5B2166%2C%201439%5D%22%5D%20%3B%0A105%20-%3E%20106%20%3B%0A107%20%5Blabel%3D%22age%20%3C%3D%2038.5%5Cngini%20%3D%200.457%5Cnsamples%20%3D%201926%5Cnvalue%20%3D%20%5B1246%2C%20680%5D%22%5D%20%3B%0A106%20-%3E%20107%20%3B%0A108%20%5Blabel%3D%22gini%20%3D%200.403%5Cnsamples%20%3D%20746%5Cnvalue%20%3D%20%5B537%2C%20209%5D%22%5D%20%3B%0A107%20-%3E%20108%20%3B%0A109%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%201180%5Cnvalue%20%3D%20%5B709%2C%20471%5D%22%5D%20%3B%0A107%20-%3E%20109%20%3B%0A110%20%5Blabel%3D%22age%20%3C%3D%2038.5%5Cngini%20%3D%200.495%5Cnsamples%20%3D%201679%5Cnvalue%20%3D%20%5B920%2C%20759%5D%22%5D%20%3B%0A106%20-%3E%20110%20%3B%0A111%20%5Blabel%3D%22gini%20%3D%200.458%5Cnsamples%20%3D%20598%5Cnvalue%20%3D%20%5B386%2C%20212%5D%22%5D%20%3B%0A110%20-%3E%20111%20%3B%0A112%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%201081%5Cnvalue%20%3D%20%5B534%2C%20547%5D%22%5D%20%3B%0A110%20-%3E%20112%20%3B%0A113%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.255%5Cnsamples%20%3D%20694%5Cnvalue%20%3D%20%5B590%2C%20104%5D%22%5D%20%3B%0A105%20-%3E%20113%20%3B%0A114%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.5%5Cngini%20%3D%200.17%5Cnsamples%20%3D%20319%5Cnvalue%20%3D%20%5B289%2C%2030%5D%22%5D%20%3B%0A113%20-%3E%20114%20%3B%0A115%20%5Blabel%3D%22gini%20%3D%200.096%5Cnsamples%20%3D%20158%5Cnvalue%20%3D%20%5B150%2C%208%5D%22%5D%20%3B%0A114%20-%3E%20115%20%3B%0A116%20%5Blabel%3D%22gini%20%3D%200.236%5Cnsamples%20%3D%20161%5Cnvalue%20%3D%20%5B139%2C%2022%5D%22%5D%20%3B%0A114%20-%3E%20116%20%3B%0A117%20%5Blabel%3D%22age%20%3C%3D%2037.5%5Cngini%20%3D%200.317%5Cnsamples%20%3D%20375%5Cnvalue%20%3D%20%5B301%2C%2074%5D%22%5D%20%3B%0A113%20-%3E%20117%20%3B%0A118%20%5Blabel%3D%22gini%20%3D%200.393%5Cnsamples%20%3D%20104%5Cnvalue%20%3D%20%5B76%2C%2028%5D%22%5D%20%3B%0A117%20-%3E%20118%20%3B%0A119%20%5Blabel%3D%22gini%20%3D%200.282%5Cnsamples%20%3D%20271%5Cnvalue%20%3D%20%5B225%2C%2046%5D%22%5D%20%3B%0A117%20-%3E%20119%20%3B%0A120%20%5Blabel%3D%22age%20%3C%3D%2028.5%5Cngini%20%3D%200.499%5Cnsamples%20%3D%208476%5Cnvalue%20%3D%20%5B4413%2C%204063%5D%22%5D%20%3B%0A0%20-%3E%20120%20%5Blabeldistance%3D2.5%2C%20labelangle%3D-45%2C%20headlabel%3D%22False%22%5D%20%3B%0A121%20%5Blabel%3D%22age%20%3C%3D%2027.5%5Cngini%20%3D%200.197%5Cnsamples%20%3D%201473%5Cnvalue%20%3D%20%5B1310%2C%20163%5D%22%5D%20%3B%0A120%20-%3E%20121%20%3B%0A122%20%5Blabel%3D%22hours-per-week%20%3C%3D%2053.5%5Cngini%20%3D%200.143%5Cnsamples%20%3D%201242%5Cnvalue%20%3D%20%5B1146%2C%2096%5D%22%5D%20%3B%0A121%20-%3E%20122%20%3B%0A123%20%5Blabel%3D%22age%20%3C%3D%2024.5%5Cngini%20%3D%200.121%5Cnsamples%20%3D%201148%5Cnvalue%20%3D%20%5B1074%2C%2074%5D%22%5D%20%3B%0A122%20-%3E%20123%20%3B%0A124%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.053%5Cnsamples%20%3D%20515%5Cnvalue%20%3D%20%5B501%2C%2014%5D%22%5D%20%3B%0A123%20-%3E%20124%20%3B%0A125%20%5Blabel%3D%22age%20%3C%3D%2021.5%5Cngini%20%3D%200.047%5Cnsamples%20%3D%20494%5Cnvalue%20%3D%20%5B482%2C%2012%5D%22%5D%20%3B%0A124%20-%3E%20125%20%3B%0A126%20%5Blabel%3D%22gini%20%3D%200.165%5Cnsamples%20%3D%2011%5Cnvalue%20%3D%20%5B10%2C%201%5D%22%5D%20%3B%0A125%20-%3E%20126%20%3B%0A127%20%5Blabel%3D%22gini%20%3D%200.045%5Cnsamples%20%3D%20483%5Cnvalue%20%3D%20%5B472%2C%2011%5D%22%5D%20%3B%0A125%20-%3E%20127%20%3B%0A128%20%5Blabel%3D%22hours-per-week%20%3C%3D%2042.5%5Cngini%20%3D%200.172%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B19%2C%202%5D%22%5D%20%3B%0A124%20-%3E%20128%20%3B%0A129%20%5Blabel%3D%22gini%20%3D%200.105%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B17%2C%201%5D%22%5D%20%3B%0A128%20-%3E%20129%20%3B%0A130%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%203%5Cnvalue%20%3D%20%5B2%2C%201%5D%22%5D%20%3B%0A128%20-%3E%20130%20%3B%0A131%20%5Blabel%3D%22hours-per-week%20%3C%3D%2037.5%5Cngini%20%3D%200.172%5Cnsamples%20%3D%20633%5Cnvalue%20%3D%20%5B573%2C%2060%5D%22%5D%20%3B%0A123%20-%3E%20131%20%3B%0A132%20%5Blabel%3D%22hours-per-week%20%3C%3D%2031.0%5Cngini%20%3D%200.045%5Cnsamples%20%3D%20130%5Cnvalue%20%3D%20%5B127%2C%203%5D%22%5D%20%3B%0A131%20-%3E%20132%20%3B%0A133%20%5Blabel%3D%22gini%20%3D%200.066%5Cnsamples%20%3D%2088%5Cnvalue%20%3D%20%5B85%2C%203%5D%22%5D%20%3B%0A132%20-%3E%20133%20%3B%0A134%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2042%5Cnvalue%20%3D%20%5B42%2C%200%5D%22%5D%20%3B%0A132%20-%3E%20134%20%3B%0A135%20%5Blabel%3D%22educational-num%20%3C%3D%2014.5%5Cngini%20%3D%200.201%5Cnsamples%20%3D%20503%5Cnvalue%20%3D%20%5B446%2C%2057%5D%22%5D%20%3B%0A131%20-%3E%20135%20%3B%0A136%20%5Blabel%3D%22gini%20%3D%200.194%5Cnsamples%20%3D%20496%5Cnvalue%20%3D%20%5B442%2C%2054%5D%22%5D%20%3B%0A135%20-%3E%20136%20%3B%0A137%20%5Blabel%3D%22gini%20%3D%200.49%5Cnsamples%20%3D%207%5Cnvalue%20%3D%20%5B4%2C%203%5D%22%5D%20%3B%0A135%20-%3E%20137%20%3B%0A138%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.359%5Cnsamples%20%3D%2094%5Cnvalue%20%3D%20%5B72%2C%2022%5D%22%5D%20%3B%0A122%20-%3E%20138%20%3B%0A139%20%5Blabel%3D%22hours-per-week%20%3C%3D%2073.5%5Cngini%20%3D%200.404%5Cnsamples%20%3D%2064%5Cnvalue%20%3D%20%5B46%2C%2018%5D%22%5D%20%3B%0A138%20-%3E%20139%20%3B%0A140%20%5Blabel%3D%22hours-per-week%20%3C%3D%2064.5%5Cngini%20%3D%200.357%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B43%2C%2013%5D%22%5D%20%3B%0A139%20-%3E%20140%20%3B%0A141%20%5Blabel%3D%22gini%20%3D%200.391%5Cnsamples%20%3D%2045%5Cnvalue%20%3D%20%5B33%2C%2012%5D%22%5D%20%3B%0A140%20-%3E%20141%20%3B%0A142%20%5Blabel%3D%22gini%20%3D%200.165%5Cnsamples%20%3D%2011%5Cnvalue%20%3D%20%5B10%2C%201%5D%22%5D%20%3B%0A140%20-%3E%20142%20%3B%0A143%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.469%5Cnsamples%20%3D%208%5Cnvalue%20%3D%20%5B3%2C%205%5D%22%5D%20%3B%0A139%20-%3E%20143%20%3B%0A144%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%205%5Cnvalue%20%3D%20%5B1%2C%204%5D%22%5D%20%3B%0A143%20-%3E%20144%20%3B%0A145%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%203%5Cnvalue%20%3D%20%5B2%2C%201%5D%22%5D%20%3B%0A143%20-%3E%20145%20%3B%0A146%20%5Blabel%3D%22age%20%3C%3D%2023.5%5Cngini%20%3D%200.231%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B26%2C%204%5D%22%5D%20%3B%0A138%20-%3E%20146%20%3B%0A147%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%206%5Cnvalue%20%3D%20%5B6%2C%200%5D%22%5D%20%3B%0A146%20-%3E%20147%20%3B%0A148%20%5Blabel%3D%22hours-per-week%20%3C%3D%2062.5%5Cngini%20%3D%200.278%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B20%2C%204%5D%22%5D%20%3B%0A146%20-%3E%20148%20%3B%0A149%20%5Blabel%3D%22gini%20%3D%200.36%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B13%2C%204%5D%22%5D%20%3B%0A148%20-%3E%20149%20%3B%0A150%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%207%5Cnvalue%20%3D%20%5B7%2C%200%5D%22%5D%20%3B%0A148%20-%3E%20150%20%3B%0A151%20%5Blabel%3D%22hours-per-week%20%3C%3D%2041.0%5Cngini%20%3D%200.412%5Cnsamples%20%3D%20231%5Cnvalue%20%3D%20%5B164%2C%2067%5D%22%5D%20%3B%0A121%20-%3E%20151%20%3B%0A152%20%5Blabel%3D%22hours-per-week%20%3C%3D%2019.0%5Cngini%20%3D%200.344%5Cnsamples%20%3D%20118%5Cnvalue%20%3D%20%5B92%2C%2026%5D%22%5D%20%3B%0A151%20-%3E%20152%20%3B%0A153%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2011%5Cnvalue%20%3D%20%5B11%2C%200%5D%22%5D%20%3B%0A152%20-%3E%20153%20%3B%0A154%20%5Blabel%3D%22hours-per-week%20%3C%3D%2032.5%5Cngini%20%3D%200.368%5Cnsamples%20%3D%20107%5Cnvalue%20%3D%20%5B81%2C%2026%5D%22%5D%20%3B%0A152%20-%3E%20154%20%3B%0A155%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.48%5Cnsamples%20%3D%2010%5Cnvalue%20%3D%20%5B6%2C%204%5D%22%5D%20%3B%0A154%20-%3E%20155%20%3B%0A156%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%204%5Cnvalue%20%3D%20%5B4%2C%200%5D%22%5D%20%3B%0A155%20-%3E%20156%20%3B%0A157%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%206%5Cnvalue%20%3D%20%5B2%2C%204%5D%22%5D%20%3B%0A155%20-%3E%20157%20%3B%0A158%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.351%5Cnsamples%20%3D%2097%5Cnvalue%20%3D%20%5B75%2C%2022%5D%22%5D%20%3B%0A154%20-%3E%20158%20%3B%0A159%20%5Blabel%3D%22gini%20%3D%200.419%5Cnsamples%20%3D%2057%5Cnvalue%20%3D%20%5B40%2C%2017%5D%22%5D%20%3B%0A158%20-%3E%20159%20%3B%0A160%20%5Blabel%3D%22gini%20%3D%200.219%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B35%2C%205%5D%22%5D%20%3B%0A158%20-%3E%20160%20%3B%0A161%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.462%5Cnsamples%20%3D%20113%5Cnvalue%20%3D%20%5B72%2C%2041%5D%22%5D%20%3B%0A151%20-%3E%20161%20%3B%0A162%20%5Blabel%3D%22hours-per-week%20%3C%3D%2062.5%5Cngini%20%3D%200.479%5Cnsamples%20%3D%2083%5Cnvalue%20%3D%20%5B50%2C%2033%5D%22%5D%20%3B%0A161%20-%3E%20162%20%3B%0A163%20%5Blabel%3D%22hours-per-week%20%3C%3D%2056.0%5Cngini%20%3D%200.471%5Cnsamples%20%3D%2079%5Cnvalue%20%3D%20%5B49%2C%2030%5D%22%5D%20%3B%0A162%20-%3E%20163%20%3B%0A164%20%5Blabel%3D%22gini%20%3D%200.464%5Cnsamples%20%3D%2071%5Cnvalue%20%3D%20%5B45%2C%2026%5D%22%5D%20%3B%0A163%20-%3E%20164%20%3B%0A165%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%208%5Cnvalue%20%3D%20%5B4%2C%204%5D%22%5D%20%3B%0A163%20-%3E%20165%20%3B%0A166%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%204%5Cnvalue%20%3D%20%5B1%2C%203%5D%22%5D%20%3B%0A162%20-%3E%20166%20%3B%0A167%20%5Blabel%3D%22hours-per-week%20%3C%3D%2057.5%5Cngini%20%3D%200.391%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B22%2C%208%5D%22%5D%20%3B%0A161%20-%3E%20167%20%3B%0A168%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.363%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B16%2C%205%5D%22%5D%20%3B%0A167%20-%3E%20168%20%3B%0A169%20%5Blabel%3D%22gini%20%3D%200.415%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B12%2C%205%5D%22%5D%20%3B%0A168%20-%3E%20169%20%3B%0A170%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%204%5Cnvalue%20%3D%20%5B4%2C%200%5D%22%5D%20%3B%0A168%20-%3E%20170%20%3B%0A171%20%5Blabel%3D%22hours-per-week%20%3C%3D%2062.5%5Cngini%20%3D%200.444%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B6%2C%203%5D%22%5D%20%3B%0A167%20-%3E%20171%20%3B%0A172%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%204%5Cnvalue%20%3D%20%5B2%2C%202%5D%22%5D%20%3B%0A171%20-%3E%20172%20%3B%0A173%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%205%5Cnvalue%20%3D%20%5B4%2C%201%5D%22%5D%20%3B%0A171%20-%3E%20173%20%3B%0A174%20%5Blabel%3D%22hours-per-week%20%3C%3D%2042.5%5Cngini%20%3D%200.494%5Cnsamples%20%3D%207003%5Cnvalue%20%3D%20%5B3103%2C%203900%5D%22%5D%20%3B%0A120%20-%3E%20174%20%3B%0A175%20%5Blabel%3D%22hours-per-week%20%3C%3D%2030.5%5Cngini%20%3D%200.498%5Cnsamples%20%3D%203853%5Cnvalue%20%3D%20%5B2046%2C%201807%5D%22%5D%20%3B%0A174%20-%3E%20175%20%3B%0A176%20%5Blabel%3D%22age%20%3C%3D%2034.5%5Cngini%20%3D%200.426%5Cnsamples%20%3D%20591%5Cnvalue%20%3D%20%5B409%2C%20182%5D%22%5D%20%3B%0A175%20-%3E%20176%20%3B%0A177%20%5Blabel%3D%22hours-per-week%20%3C%3D%2026.0%5Cngini%20%3D%200.305%5Cnsamples%20%3D%20101%5Cnvalue%20%3D%20%5B82%2C%2019%5D%22%5D%20%3B%0A176%20-%3E%20177%20%3B%0A178%20%5Blabel%3D%22hours-per-week%20%3C%3D%2022.5%5Cngini%20%3D%200.346%5Cnsamples%20%3D%2081%5Cnvalue%20%3D%20%5B63%2C%2018%5D%22%5D%20%3B%0A177%20-%3E%20178%20%3B%0A179%20%5Blabel%3D%22gini%20%3D%200.252%5Cnsamples%20%3D%2054%5Cnvalue%20%3D%20%5B46%2C%208%5D%22%5D%20%3B%0A178%20-%3E%20179%20%3B%0A180%20%5Blabel%3D%22gini%20%3D%200.466%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B17%2C%2010%5D%22%5D%20%3B%0A178%20-%3E%20180%20%3B%0A181%20%5Blabel%3D%22age%20%3C%3D%2033.5%5Cngini%20%3D%200.095%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B19%2C%201%5D%22%5D%20%3B%0A177%20-%3E%20181%20%3B%0A182%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B14%2C%200%5D%22%5D%20%3B%0A181%20-%3E%20182%20%3B%0A183%20%5Blabel%3D%22gini%20%3D%200.278%5Cnsamples%20%3D%206%5Cnvalue%20%3D%20%5B5%2C%201%5D%22%5D%20%3B%0A181%20-%3E%20183%20%3B%0A184%20%5Blabel%3D%22educational-num%20%3C%3D%2015.5%5Cngini%20%3D%200.444%5Cnsamples%20%3D%20490%5Cnvalue%20%3D%20%5B327%2C%20163%5D%22%5D%20%3B%0A176%20-%3E%20184%20%3B%0A185%20%5Blabel%3D%22age%20%3C%3D%2067.5%5Cngini%20%3D%200.433%5Cnsamples%20%3D%20454%5Cnvalue%20%3D%20%5B310%2C%20144%5D%22%5D%20%3B%0A184%20-%3E%20185%20%3B%0A186%20%5Blabel%3D%22gini%20%3D%200.452%5Cnsamples%20%3D%20365%5Cnvalue%20%3D%20%5B239%2C%20126%5D%22%5D%20%3B%0A185%20-%3E%20186%20%3B%0A187%20%5Blabel%3D%22gini%20%3D%200.323%5Cnsamples%20%3D%2089%5Cnvalue%20%3D%20%5B71%2C%2018%5D%22%5D%20%3B%0A185%20-%3E%20187%20%3B%0A188%20%5Blabel%3D%22age%20%3C%3D%2062.0%5Cngini%20%3D%200.498%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B17%2C%2019%5D%22%5D%20%3B%0A184%20-%3E%20188%20%3B%0A189%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B14%2C%207%5D%22%5D%20%3B%0A188%20-%3E%20189%20%3B%0A190%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B3%2C%2012%5D%22%5D%20%3B%0A188%20-%3E%20190%20%3B%0A191%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.5%5Cnsamples%20%3D%203262%5Cnvalue%20%3D%20%5B1637%2C%201625%5D%22%5D%20%3B%0A175%20-%3E%20191%20%3B%0A192%20%5Blabel%3D%22age%20%3C%3D%2037.5%5Cngini%20%3D%200.483%5Cnsamples%20%3D%202278%5Cnvalue%20%3D%20%5B929%2C%201349%5D%22%5D%20%3B%0A191%20-%3E%20192%20%3B%0A193%20%5Blabel%3D%22age%20%3C%3D%2033.5%5Cngini%20%3D%200.496%5Cnsamples%20%3D%20680%5Cnvalue%20%3D%20%5B371%2C%20309%5D%22%5D%20%3B%0A192%20-%3E%20193%20%3B%0A194%20%5Blabel%3D%22gini%20%3D%200.482%5Cnsamples%20%3D%20375%5Cnvalue%20%3D%20%5B223%2C%20152%5D%22%5D%20%3B%0A193%20-%3E%20194%20%3B%0A195%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%20305%5Cnvalue%20%3D%20%5B148%2C%20157%5D%22%5D%20%3B%0A193%20-%3E%20195%20%3B%0A196%20%5Blabel%3D%22age%20%3C%3D%2042.5%5Cngini%20%3D%200.455%5Cnsamples%20%3D%201598%5Cnvalue%20%3D%20%5B558%2C%201040%5D%22%5D%20%3B%0A192%20-%3E%20196%20%3B%0A197%20%5Blabel%3D%22gini%20%3D%200.488%5Cnsamples%20%3D%20404%5Cnvalue%20%3D%20%5B171%2C%20233%5D%22%5D%20%3B%0A196%20-%3E%20197%20%3B%0A198%20%5Blabel%3D%22gini%20%3D%200.438%5Cnsamples%20%3D%201194%5Cnvalue%20%3D%20%5B387%2C%20807%5D%22%5D%20%3B%0A196%20-%3E%20198%20%3B%0A199%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.404%5Cnsamples%20%3D%20984%5Cnvalue%20%3D%20%5B708%2C%20276%5D%22%5D%20%3B%0A191%20-%3E%20199%20%3B%0A200%20%5Blabel%3D%22hours-per-week%20%3C%3D%2035.5%5Cngini%20%3D%200.365%5Cnsamples%20%3D%20649%5Cnvalue%20%3D%20%5B493%2C%20156%5D%22%5D%20%3B%0A199%20-%3E%20200%20%3B%0A201%20%5Blabel%3D%22gini%20%3D%200.456%5Cnsamples%20%3D%2074%5Cnvalue%20%3D%20%5B48%2C%2026%5D%22%5D%20%3B%0A200%20-%3E%20201%20%3B%0A202%20%5Blabel%3D%22gini%20%3D%200.35%5Cnsamples%20%3D%20575%5Cnvalue%20%3D%20%5B445%2C%20130%5D%22%5D%20%3B%0A200%20-%3E%20202%20%3B%0A203%20%5Blabel%3D%22educational-num%20%3C%3D%2015.5%5Cngini%20%3D%200.46%5Cnsamples%20%3D%20335%5Cnvalue%20%3D%20%5B215%2C%20120%5D%22%5D%20%3B%0A199%20-%3E%20203%20%3B%0A204%20%5Blabel%3D%22gini%20%3D%200.445%5Cnsamples%20%3D%20302%5Cnvalue%20%3D%20%5B201%2C%20101%5D%22%5D%20%3B%0A203%20-%3E%20204%20%3B%0A205%20%5Blabel%3D%22gini%20%3D%200.489%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B14%2C%2019%5D%22%5D%20%3B%0A203%20-%3E%20205%20%3B%0A206%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.446%5Cnsamples%20%3D%203150%5Cnvalue%20%3D%20%5B1057%2C%202093%5D%22%5D%20%3B%0A174%20-%3E%20206%20%3B%0A207%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.411%5Cnsamples%20%3D%202601%5Cnvalue%20%3D%20%5B751%2C%201850%5D%22%5D%20%3B%0A206%20-%3E%20207%20%3B%0A208%20%5Blabel%3D%22age%20%3C%3D%2043.5%5Cngini%20%3D%200.453%5Cnsamples%20%3D%201489%5Cnvalue%20%3D%20%5B516%2C%20973%5D%22%5D%20%3B%0A207%20-%3E%20208%20%3B%0A209%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.0%5Cngini%20%3D%200.474%5Cnsamples%20%3D%20886%5Cnvalue%20%3D%20%5B342%2C%20544%5D%22%5D%20%3B%0A208%20-%3E%20209%20%3B%0A210%20%5Blabel%3D%22gini%20%3D%200.492%5Cnsamples%20%3D%20248%5Cnvalue%20%3D%20%5B108%2C%20140%5D%22%5D%20%3B%0A209%20-%3E%20210%20%3B%0A211%20%5Blabel%3D%22gini%20%3D%200.465%5Cnsamples%20%3D%20638%5Cnvalue%20%3D%20%5B234%2C%20404%5D%22%5D%20%3B%0A209%20-%3E%20211%20%3B%0A212%20%5Blabel%3D%22hours-per-week%20%3C%3D%2073.5%5Cngini%20%3D%200.411%5Cnsamples%20%3D%20603%5Cnvalue%20%3D%20%5B174%2C%20429%5D%22%5D%20%3B%0A208%20-%3E%20212%20%3B%0A213%20%5Blabel%3D%22gini%20%3D%200.403%5Cnsamples%20%3D%20590%5Cnvalue%20%3D%20%5B165%2C%20425%5D%22%5D%20%3B%0A212%20-%3E%20213%20%3B%0A214%20%5Blabel%3D%22gini%20%3D%200.426%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B9%2C%204%5D%22%5D%20%3B%0A212%20-%3E%20214%20%3B%0A215%20%5Blabel%3D%22age%20%3C%3D%2032.5%5Cngini%20%3D%200.333%5Cnsamples%20%3D%201112%5Cnvalue%20%3D%20%5B235%2C%20877%5D%22%5D%20%3B%0A207%20-%3E%20215%20%3B%0A216%20%5Blabel%3D%22hours-per-week%20%3C%3D%2052.5%5Cngini%20%3D%200.486%5Cnsamples%20%3D%2096%5Cnvalue%20%3D%20%5B40%2C%2056%5D%22%5D%20%3B%0A215%20-%3E%20216%20%3B%0A217%20%5Blabel%3D%22gini%20%3D%200.497%5Cnsamples%20%3D%2050%5Cnvalue%20%3D%20%5B23%2C%2027%5D%22%5D%20%3B%0A216%20-%3E%20217%20%3B%0A218%20%5Blabel%3D%22gini%20%3D%200.466%5Cnsamples%20%3D%2046%5Cnvalue%20%3D%20%5B17%2C%2029%5D%22%5D%20%3B%0A216%20-%3E%20218%20%3B%0A219%20%5Blabel%3D%22educational-num%20%3C%3D%2014.5%5Cngini%20%3D%200.31%5Cnsamples%20%3D%201016%5Cnvalue%20%3D%20%5B195%2C%20821%5D%22%5D%20%3B%0A215%20-%3E%20219%20%3B%0A220%20%5Blabel%3D%22gini%20%3D%200.364%5Cnsamples%20%3D%20569%5Cnvalue%20%3D%20%5B136%2C%20433%5D%22%5D%20%3B%0A219%20-%3E%20220%20%3B%0A221%20%5Blabel%3D%22gini%20%3D%200.229%5Cnsamples%20%3D%20447%5Cnvalue%20%3D%20%5B59%2C%20388%5D%22%5D%20%3B%0A219%20-%3E%20221%20%3B%0A222%20%5Blabel%3D%22educational-num%20%3C%3D%2014.5%5Cngini%20%3D%200.493%5Cnsamples%20%3D%20549%5Cnvalue%20%3D%20%5B306%2C%20243%5D%22%5D%20%3B%0A206%20-%3E%20222%20%3B%0A223%20%5Blabel%3D%22hours-per-week%20%3C%3D%2062.5%5Cngini%20%3D%200.476%5Cnsamples%20%3D%20483%5Cnvalue%20%3D%20%5B294%2C%20189%5D%22%5D%20%3B%0A222%20-%3E%20223%20%3B%0A224%20%5Blabel%3D%22age%20%3C%3D%2058.5%5Cngini%20%3D%200.482%5Cnsamples%20%3D%20447%5Cnvalue%20%3D%20%5B266%2C%20181%5D%22%5D%20%3B%0A223%20-%3E%20224%20%3B%0A225%20%5Blabel%3D%22gini%20%3D%200.478%5Cnsamples%20%3D%20427%5Cnvalue%20%3D%20%5B258%2C%20169%5D%22%5D%20%3B%0A224%20-%3E%20225%20%3B%0A226%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B8%2C%2012%5D%22%5D%20%3B%0A224%20-%3E%20226%20%3B%0A227%20%5Blabel%3D%22hours-per-week%20%3C%3D%2067.5%5Cngini%20%3D%200.346%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B28%2C%208%5D%22%5D%20%3B%0A223%20-%3E%20227%20%3B%0A228%20%5Blabel%3D%22gini%20%3D%200.219%5Cnsamples%20%3D%208%5Cnvalue%20%3D%20%5B7%2C%201%5D%22%5D%20%3B%0A227%20-%3E%20228%20%3B%0A229%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B21%2C%207%5D%22%5D%20%3B%0A227%20-%3E%20229%20%3B%0A230%20%5Blabel%3D%22age%20%3C%3D%2029.5%5Cngini%20%3D%200.298%5Cnsamples%20%3D%2066%5Cnvalue%20%3D%20%5B12%2C%2054%5D%22%5D%20%3B%0A222%20-%3E%20230%20%3B%0A231%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%203%5Cnvalue%20%3D%20%5B2%2C%201%5D%22%5D%20%3B%0A230%20-%3E%20231%20%3B%0A232%20%5Blabel%3D%22hours-per-week%20%3C%3D%2089.5%5Cngini%20%3D%200.267%5Cnsamples%20%3D%2063%5Cnvalue%20%3D%20%5B10%2C%2053%5D%22%5D%20%3B%0A230%20-%3E%20232%20%3B%0A233%20%5Blabel%3D%22gini%20%3D%200.209%5Cnsamples%20%3D%2059%5Cnvalue%20%3D%20%5B7%2C%2052%5D%22%5D%20%3B%0A232%20-%3E%20233%20%3B%0A234%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%204%5Cnvalue%20%3D%20%5B3%2C%201%5D%22%5D%20%3B%0A232%20-%3E%20234%20%3B%0A%7D)

[Картинка](https://dreampuf.github.io/GraphvizOnline/#digraph%20Tree%20%7B%0Anode%20%5Bshape%3Dbox%5D%20%3B%0A0%20%5Blabel%3D%22educational-num%20%3C%3D%2012.5%5Cngini%20%3D%200.365%5Cnsamples%20%3D%2034189%5Cnvalue%20%3D%20%5B25991%2C%208198%5D%22%5D%20%3B%0A1%20%5Blabel%3D%22age%20%3C%3D%2030.5%5Cngini%20%3D%200.27%5Cnsamples%20%3D%2025713%5Cnvalue%20%3D%20%5B21578%2C%204135%5D%22%5D%20%3B%0A0%20-%3E%201%20%5Blabeldistance%3D2.5%2C%20labelangle%3D45%2C%20headlabel%3D%22True%22%5D%20%3B%0A2%20%5Blabel%3D%22hours-per-week%20%3C%3D%2041.5%5Cngini%20%3D%200.079%5Cnsamples%20%3D%209001%5Cnvalue%20%3D%20%5B8632%2C%20369%5D%22%5D%20%3B%0A1%20-%3E%202%20%3B%0A3%20%5Blabel%3D%22age%20%3C%3D%2026.5%5Cngini%20%3D%200.057%5Cnsamples%20%3D%207453%5Cnvalue%20%3D%20%5B7235%2C%20218%5D%22%5D%20%3B%0A2%20-%3E%203%20%3B%0A4%20%5Blabel%3D%22age%20%3C%3D%2024.5%5Cngini%20%3D%200.022%5Cnsamples%20%3D%205587%5Cnvalue%20%3D%20%5B5525%2C%2062%5D%22%5D%20%3B%0A3%20-%3E%204%20%3B%0A5%20%5Blabel%3D%22age%20%3C%3D%2021.5%5Cngini%20%3D%200.011%5Cnsamples%20%3D%204684%5Cnvalue%20%3D%20%5B4658%2C%2026%5D%22%5D%20%3B%0A4%20-%3E%205%20%3B%0A6%20%5Blabel%3D%22hours-per-week%20%3C%3D%2039.5%5Cngini%20%3D%200.003%5Cnsamples%20%3D%202999%5Cnvalue%20%3D%20%5B2995%2C%204%5D%22%5D%20%3B%0A5%20-%3E%206%20%3B%0A7%20%5Blabel%3D%22gini%20%3D%200.001%5Cnsamples%20%3D%202034%5Cnvalue%20%3D%20%5B2033%2C%201%5D%22%5D%20%3B%0A6%20-%3E%207%20%3B%0A8%20%5Blabel%3D%22gini%20%3D%200.006%5Cnsamples%20%3D%20965%5Cnvalue%20%3D%20%5B962%2C%203%5D%22%5D%20%3B%0A6%20-%3E%208%20%3B%0A9%20%5Blabel%3D%22race%20%3C%3D%201.5%5Cngini%20%3D%200.026%5Cnsamples%20%3D%201685%5Cnvalue%20%3D%20%5B1663%2C%2022%5D%22%5D%20%3B%0A5%20-%3E%209%20%3B%0A10%20%5Blabel%3D%22gini%20%3D%200.022%5Cnsamples%20%3D%201583%5Cnvalue%20%3D%20%5B1565%2C%2018%5D%22%5D%20%3B%0A9%20-%3E%2010%20%3B%0A11%20%5Blabel%3D%22gini%20%3D%200.075%5Cnsamples%20%3D%20102%5Cnvalue%20%3D%20%5B98%2C%204%5D%22%5D%20%3B%0A9%20-%3E%2011%20%3B%0A12%20%5Blabel%3D%22educational-num%20%3C%3D%2010.5%5Cngini%20%3D%200.077%5Cnsamples%20%3D%20903%5Cnvalue%20%3D%20%5B867%2C%2036%5D%22%5D%20%3B%0A4%20-%3E%2012%20%3B%0A13%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.062%5Cnsamples%20%3D%20811%5Cnvalue%20%3D%20%5B785%2C%2026%5D%22%5D%20%3B%0A12%20-%3E%2013%20%3B%0A14%20%5Blabel%3D%22gini%20%3D%200.085%5Cnsamples%20%3D%20497%5Cnvalue%20%3D%20%5B475%2C%2022%5D%22%5D%20%3B%0A13%20-%3E%2014%20%3B%0A15%20%5Blabel%3D%22gini%20%3D%200.025%5Cnsamples%20%3D%20314%5Cnvalue%20%3D%20%5B310%2C%204%5D%22%5D%20%3B%0A13%20-%3E%2015%20%3B%0A16%20%5Blabel%3D%22race%20%3C%3D%200.5%5Cngini%20%3D%200.194%5Cnsamples%20%3D%2092%5Cnvalue%20%3D%20%5B82%2C%2010%5D%22%5D%20%3B%0A12%20-%3E%2016%20%3B%0A17%20%5Blabel%3D%22gini%20%3D%200.202%5Cnsamples%20%3D%2079%5Cnvalue%20%3D%20%5B70%2C%209%5D%22%5D%20%3B%0A16%20-%3E%2017%20%3B%0A18%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B12%2C%201%5D%22%5D%20%3B%0A16%20-%3E%2018%20%3B%0A19%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.153%5Cnsamples%20%3D%201866%5Cnvalue%20%3D%20%5B1710%2C%20156%5D%22%5D%20%3B%0A3%20-%3E%2019%20%3B%0A20%20%5Blabel%3D%22educational-num%20%3C%3D%207.5%5Cngini%20%3D%200.118%5Cnsamples%20%3D%201143%5Cnvalue%20%3D%20%5B1071%2C%2072%5D%22%5D%20%3B%0A19%20-%3E%2020%20%3B%0A21%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.04%5Cnsamples%20%3D%20243%5Cnvalue%20%3D%20%5B238%2C%205%5D%22%5D%20%3B%0A20%20-%3E%2021%20%3B%0A22%20%5Blabel%3D%22gini%20%3D%200.057%5Cnsamples%20%3D%20171%5Cnvalue%20%3D%20%5B166%2C%205%5D%22%5D%20%3B%0A21%20-%3E%2022%20%3B%0A23%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2072%5Cnvalue%20%3D%20%5B72%2C%200%5D%22%5D%20%3B%0A21%20-%3E%2023%20%3B%0A24%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.138%5Cnsamples%20%3D%20900%5Cnvalue%20%3D%20%5B833%2C%2067%5D%22%5D%20%3B%0A20%20-%3E%2024%20%3B%0A25%20%5Blabel%3D%22gini%20%3D%200.168%5Cnsamples%20%3D%20584%5Cnvalue%20%3D%20%5B530%2C%2054%5D%22%5D%20%3B%0A24%20-%3E%2025%20%3B%0A26%20%5Blabel%3D%22gini%20%3D%200.079%5Cnsamples%20%3D%20316%5Cnvalue%20%3D%20%5B303%2C%2013%5D%22%5D%20%3B%0A24%20-%3E%2026%20%3B%0A27%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.205%5Cnsamples%20%3D%20723%5Cnvalue%20%3D%20%5B639%2C%2084%5D%22%5D%20%3B%0A19%20-%3E%2027%20%3B%0A28%20%5Blabel%3D%22hours-per-week%20%3C%3D%2031.0%5Cngini%20%3D%200.246%5Cnsamples%20%3D%20397%5Cnvalue%20%3D%20%5B340%2C%2057%5D%22%5D%20%3B%0A27%20-%3E%2028%20%3B%0A29%20%5Blabel%3D%22gini%20%3D%200.046%5Cnsamples%20%3D%2042%5Cnvalue%20%3D%20%5B41%2C%201%5D%22%5D%20%3B%0A28%20-%3E%2029%20%3B%0A30%20%5Blabel%3D%22gini%20%3D%200.266%5Cnsamples%20%3D%20355%5Cnvalue%20%3D%20%5B299%2C%2056%5D%22%5D%20%3B%0A28%20-%3E%2030%20%3B%0A31%20%5Blabel%3D%22hours-per-week%20%3C%3D%2037.5%5Cngini%20%3D%200.152%5Cnsamples%20%3D%20326%5Cnvalue%20%3D%20%5B299%2C%2027%5D%22%5D%20%3B%0A27%20-%3E%2031%20%3B%0A32%20%5Blabel%3D%22gini%20%3D%200.225%5Cnsamples%20%3D%2093%5Cnvalue%20%3D%20%5B81%2C%2012%5D%22%5D%20%3B%0A31%20-%3E%2032%20%3B%0A33%20%5Blabel%3D%22gini%20%3D%200.12%5Cnsamples%20%3D%20233%5Cnvalue%20%3D%20%5B218%2C%2015%5D%22%5D%20%3B%0A31%20-%3E%2033%20%3B%0A34%20%5Blabel%3D%22age%20%3C%3D%2024.5%5Cngini%20%3D%200.176%5Cnsamples%20%3D%201548%5Cnvalue%20%3D%20%5B1397%2C%20151%5D%22%5D%20%3B%0A2%20-%3E%2034%20%3B%0A35%20%5Blabel%3D%22age%20%3C%3D%2023.5%5Cngini%20%3D%200.061%5Cnsamples%20%3D%20567%5Cnvalue%20%3D%20%5B549%2C%2018%5D%22%5D%20%3B%0A34%20-%3E%2035%20%3B%0A36%20%5Blabel%3D%22age%20%3C%3D%2021.5%5Cngini%20%3D%200.036%5Cnsamples%20%3D%20440%5Cnvalue%20%3D%20%5B432%2C%208%5D%22%5D%20%3B%0A35%20-%3E%2036%20%3B%0A37%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.009%5Cnsamples%20%3D%20228%5Cnvalue%20%3D%20%5B227%2C%201%5D%22%5D%20%3B%0A36%20-%3E%2037%20%3B%0A38%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20175%5Cnvalue%20%3D%20%5B175%2C%200%5D%22%5D%20%3B%0A37%20-%3E%2038%20%3B%0A39%20%5Blabel%3D%22gini%20%3D%200.037%5Cnsamples%20%3D%2053%5Cnvalue%20%3D%20%5B52%2C%201%5D%22%5D%20%3B%0A37%20-%3E%2039%20%3B%0A40%20%5Blabel%3D%22educational-num%20%3C%3D%208.5%5Cngini%20%3D%200.064%5Cnsamples%20%3D%20212%5Cnvalue%20%3D%20%5B205%2C%207%5D%22%5D%20%3B%0A36%20-%3E%2040%20%3B%0A41%20%5Blabel%3D%22gini%20%3D%200.227%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B20%2C%203%5D%22%5D%20%3B%0A40%20-%3E%2041%20%3B%0A42%20%5Blabel%3D%22gini%20%3D%200.041%5Cnsamples%20%3D%20189%5Cnvalue%20%3D%20%5B185%2C%204%5D%22%5D%20%3B%0A40%20-%3E%2042%20%3B%0A43%20%5Blabel%3D%22race%20%3C%3D%200.5%5Cngini%20%3D%200.145%5Cnsamples%20%3D%20127%5Cnvalue%20%3D%20%5B117%2C%2010%5D%22%5D%20%3B%0A35%20-%3E%2043%20%3B%0A44%20%5Blabel%3D%22hours-per-week%20%3C%3D%2048.5%5Cngini%20%3D%200.164%5Cnsamples%20%3D%20111%5Cnvalue%20%3D%20%5B101%2C%2010%5D%22%5D%20%3B%0A43%20-%3E%2044%20%3B%0A45%20%5Blabel%3D%22gini%20%3D%200.093%5Cnsamples%20%3D%2041%5Cnvalue%20%3D%20%5B39%2C%202%5D%22%5D%20%3B%0A44%20-%3E%2045%20%3B%0A46%20%5Blabel%3D%22gini%20%3D%200.202%5Cnsamples%20%3D%2070%5Cnvalue%20%3D%20%5B62%2C%208%5D%22%5D%20%3B%0A44%20-%3E%2046%20%3B%0A47%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B16%2C%200%5D%22%5D%20%3B%0A43%20-%3E%2047%20%3B%0A48%20%5Blabel%3D%22educational-num%20%3C%3D%2010.5%5Cngini%20%3D%200.234%5Cnsamples%20%3D%20981%5Cnvalue%20%3D%20%5B848%2C%20133%5D%22%5D%20%3B%0A34%20-%3E%2048%20%3B%0A49%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.215%5Cnsamples%20%3D%20841%5Cnvalue%20%3D%20%5B738%2C%20103%5D%22%5D%20%3B%0A48%20-%3E%2049%20%3B%0A50%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.5%5Cngini%20%3D%200.235%5Cnsamples%20%3D%20697%5Cnvalue%20%3D%20%5B602%2C%2095%5D%22%5D%20%3B%0A49%20-%3E%2050%20%3B%0A51%20%5Blabel%3D%22gini%20%3D%200.19%5Cnsamples%20%3D%20263%5Cnvalue%20%3D%20%5B235%2C%2028%5D%22%5D%20%3B%0A50%20-%3E%2051%20%3B%0A52%20%5Blabel%3D%22gini%20%3D%200.261%5Cnsamples%20%3D%20434%5Cnvalue%20%3D%20%5B367%2C%2067%5D%22%5D%20%3B%0A50%20-%3E%2052%20%3B%0A53%20%5Blabel%3D%22age%20%3C%3D%2025.5%5Cngini%20%3D%200.105%5Cnsamples%20%3D%20144%5Cnvalue%20%3D%20%5B136%2C%208%5D%22%5D%20%3B%0A49%20-%3E%2053%20%3B%0A54%20%5Blabel%3D%22gini%20%3D%200.198%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B24%2C%203%5D%22%5D%20%3B%0A53%20-%3E%2054%20%3B%0A55%20%5Blabel%3D%22gini%20%3D%200.082%5Cnsamples%20%3D%20117%5Cnvalue%20%3D%20%5B112%2C%205%5D%22%5D%20%3B%0A53%20-%3E%2055%20%3B%0A56%20%5Blabel%3D%22hours-per-week%20%3C%3D%2064.5%5Cngini%20%3D%200.337%5Cnsamples%20%3D%20140%5Cnvalue%20%3D%20%5B110%2C%2030%5D%22%5D%20%3B%0A48%20-%3E%2056%20%3B%0A57%20%5Blabel%3D%22age%20%3C%3D%2027.5%5Cngini%20%3D%200.303%5Cnsamples%20%3D%20118%5Cnvalue%20%3D%20%5B96%2C%2022%5D%22%5D%20%3B%0A56%20-%3E%2057%20%3B%0A58%20%5Blabel%3D%22gini%20%3D%200.226%5Cnsamples%20%3D%2054%5Cnvalue%20%3D%20%5B47%2C%207%5D%22%5D%20%3B%0A57%20-%3E%2058%20%3B%0A59%20%5Blabel%3D%22gini%20%3D%200.359%5Cnsamples%20%3D%2064%5Cnvalue%20%3D%20%5B49%2C%2015%5D%22%5D%20%3B%0A57%20-%3E%2059%20%3B%0A60%20%5Blabel%3D%22hours-per-week%20%3C%3D%2073.5%5Cngini%20%3D%200.463%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B14%2C%208%5D%22%5D%20%3B%0A56%20-%3E%2060%20%3B%0A61%20%5Blabel%3D%22gini%20%3D%200.473%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B5%2C%208%5D%22%5D%20%3B%0A60%20-%3E%2061%20%3B%0A62%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B9%2C%200%5D%22%5D%20%3B%0A60%20-%3E%2062%20%3B%0A63%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.349%5Cnsamples%20%3D%2016712%5Cnvalue%20%3D%20%5B12946%2C%203766%5D%22%5D%20%3B%0A1%20-%3E%2063%20%3B%0A64%20%5Blabel%3D%22educational-num%20%3C%3D%208.5%5Cngini%20%3D%200.404%5Cnsamples%20%3D%2011587%5Cnvalue%20%3D%20%5B8330%2C%203257%5D%22%5D%20%3B%0A63%20-%3E%2064%20%3B%0A65%20%5Blabel%3D%22age%20%3C%3D%2036.5%5Cngini%20%3D%200.19%5Cnsamples%20%3D%202016%5Cnvalue%20%3D%20%5B1802%2C%20214%5D%22%5D%20%3B%0A64%20-%3E%2065%20%3B%0A66%20%5Blabel%3D%22age%20%3C%3D%2032.5%5Cngini%20%3D%200.104%5Cnsamples%20%3D%20418%5Cnvalue%20%3D%20%5B395%2C%2023%5D%22%5D%20%3B%0A65%20-%3E%2066%20%3B%0A67%20%5Blabel%3D%22hours-per-week%20%3C%3D%2053.0%5Cngini%20%3D%200.157%5Cnsamples%20%3D%20151%5Cnvalue%20%3D%20%5B138%2C%2013%5D%22%5D%20%3B%0A66%20-%3E%2067%20%3B%0A68%20%5Blabel%3D%22gini%20%3D%200.097%5Cnsamples%20%3D%20137%5Cnvalue%20%3D%20%5B130%2C%207%5D%22%5D%20%3B%0A67%20-%3E%2068%20%3B%0A69%20%5Blabel%3D%22gini%20%3D%200.49%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B8%2C%206%5D%22%5D%20%3B%0A67%20-%3E%2069%20%3B%0A70%20%5Blabel%3D%22educational-num%20%3C%3D%204.5%5Cngini%20%3D%200.072%5Cnsamples%20%3D%20267%5Cnvalue%20%3D%20%5B257%2C%2010%5D%22%5D%20%3B%0A66%20-%3E%2070%20%3B%0A71%20%5Blabel%3D%22gini%20%3D%200.148%5Cnsamples%20%3D%2062%5Cnvalue%20%3D%20%5B57%2C%205%5D%22%5D%20%3B%0A70%20-%3E%2071%20%3B%0A72%20%5Blabel%3D%22gini%20%3D%200.048%5Cnsamples%20%3D%20205%5Cnvalue%20%3D%20%5B200%2C%205%5D%22%5D%20%3B%0A70%20-%3E%2072%20%3B%0A73%20%5Blabel%3D%22hours-per-week%20%3C%3D%2040.5%5Cngini%20%3D%200.21%5Cnsamples%20%3D%201598%5Cnvalue%20%3D%20%5B1407%2C%20191%5D%22%5D%20%3B%0A65%20-%3E%2073%20%3B%0A74%20%5Blabel%3D%22educational-num%20%3C%3D%205.5%5Cngini%20%3D%200.158%5Cnsamples%20%3D%201192%5Cnvalue%20%3D%20%5B1089%2C%20103%5D%22%5D%20%3B%0A73%20-%3E%2074%20%3B%0A75%20%5Blabel%3D%22gini%20%3D%200.107%5Cnsamples%20%3D%20667%5Cnvalue%20%3D%20%5B629%2C%2038%5D%22%5D%20%3B%0A74%20-%3E%2075%20%3B%0A76%20%5Blabel%3D%22gini%20%3D%200.217%5Cnsamples%20%3D%20525%5Cnvalue%20%3D%20%5B460%2C%2065%5D%22%5D%20%3B%0A74%20-%3E%2076%20%3B%0A77%20%5Blabel%3D%22hours-per-week%20%3C%3D%2059.0%5Cngini%20%3D%200.34%5Cnsamples%20%3D%20406%5Cnvalue%20%3D%20%5B318%2C%2088%5D%22%5D%20%3B%0A73%20-%3E%2077%20%3B%0A78%20%5Blabel%3D%22gini%20%3D%200.31%5Cnsamples%20%3D%20282%5Cnvalue%20%3D%20%5B228%2C%2054%5D%22%5D%20%3B%0A77%20-%3E%2078%20%3B%0A79%20%5Blabel%3D%22gini%20%3D%200.398%5Cnsamples%20%3D%20124%5Cnvalue%20%3D%20%5B90%2C%2034%5D%22%5D%20%3B%0A77%20-%3E%2079%20%3B%0A80%20%5Blabel%3D%22hours-per-week%20%3C%3D%2035.5%5Cngini%20%3D%200.434%5Cnsamples%20%3D%209571%5Cnvalue%20%3D%20%5B6528%2C%203043%5D%22%5D%20%3B%0A64%20-%3E%2080%20%3B%0A81%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.213%5Cnsamples%20%3D%20980%5Cnvalue%20%3D%20%5B861%2C%20119%5D%22%5D%20%3B%0A80%20-%3E%2081%20%3B%0A82%20%5Blabel%3D%22hours-per-week%20%3C%3D%2024.5%5Cngini%20%3D%200.15%5Cnsamples%20%3D%20552%5Cnvalue%20%3D%20%5B507%2C%2045%5D%22%5D%20%3B%0A81%20-%3E%2082%20%3B%0A83%20%5Blabel%3D%22gini%20%3D%200.085%5Cnsamples%20%3D%20248%5Cnvalue%20%3D%20%5B237%2C%2011%5D%22%5D%20%3B%0A82%20-%3E%2083%20%3B%0A84%20%5Blabel%3D%22gini%20%3D%200.199%5Cnsamples%20%3D%20304%5Cnvalue%20%3D%20%5B270%2C%2034%5D%22%5D%20%3B%0A82%20-%3E%2084%20%3B%0A85%20%5Blabel%3D%22hours-per-week%20%3C%3D%2031.0%5Cngini%20%3D%200.286%5Cnsamples%20%3D%20428%5Cnvalue%20%3D%20%5B354%2C%2074%5D%22%5D%20%3B%0A81%20-%3E%2085%20%3B%0A86%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%20300%5Cnvalue%20%3D%20%5B254%2C%2046%5D%22%5D%20%3B%0A85%20-%3E%2086%20%3B%0A87%20%5Blabel%3D%22gini%20%3D%200.342%5Cnsamples%20%3D%20128%5Cnvalue%20%3D%20%5B100%2C%2028%5D%22%5D%20%3B%0A85%20-%3E%2087%20%3B%0A88%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.449%5Cnsamples%20%3D%208591%5Cnvalue%20%3D%20%5B5667%2C%202924%5D%22%5D%20%3B%0A80%20-%3E%2088%20%3B%0A89%20%5Blabel%3D%22age%20%3C%3D%2038.5%5Cngini%20%3D%200.412%5Cnsamples%20%3D%204805%5Cnvalue%20%3D%20%5B3413%2C%201392%5D%22%5D%20%3B%0A88%20-%3E%2089%20%3B%0A90%20%5Blabel%3D%22gini%20%3D%200.325%5Cnsamples%20%3D%201745%5Cnvalue%20%3D%20%5B1388%2C%20357%5D%22%5D%20%3B%0A89%20-%3E%2090%20%3B%0A91%20%5Blabel%3D%22gini%20%3D%200.448%5Cnsamples%20%3D%203060%5Cnvalue%20%3D%20%5B2025%2C%201035%5D%22%5D%20%3B%0A89%20-%3E%2091%20%3B%0A92%20%5Blabel%3D%22age%20%3C%3D%2038.5%5Cngini%20%3D%200.482%5Cnsamples%20%3D%203786%5Cnvalue%20%3D%20%5B2254%2C%201532%5D%22%5D%20%3B%0A88%20-%3E%2092%20%3B%0A93%20%5Blabel%3D%22gini%20%3D%200.422%5Cnsamples%20%3D%201321%5Cnvalue%20%3D%20%5B922%2C%20399%5D%22%5D%20%3B%0A92%20-%3E%2093%20%3B%0A94%20%5Blabel%3D%22gini%20%3D%200.497%5Cnsamples%20%3D%202465%5Cnvalue%20%3D%20%5B1332%2C%201133%5D%22%5D%20%3B%0A92%20-%3E%2094%20%3B%0A95%20%5Blabel%3D%22educational-num%20%3C%3D%209.5%5Cngini%20%3D%200.179%5Cnsamples%20%3D%205125%5Cnvalue%20%3D%20%5B4616%2C%20509%5D%22%5D%20%3B%0A63%20-%3E%2095%20%3B%0A96%20%5Blabel%3D%22race%20%3C%3D%200.5%5Cngini%20%3D%200.136%5Cnsamples%20%3D%203101%5Cnvalue%20%3D%20%5B2874%2C%20227%5D%22%5D%20%3B%0A95%20-%3E%2096%20%3B%0A97%20%5Blabel%3D%22educational-num%20%3C%3D%208.5%5Cngini%20%3D%200.155%5Cnsamples%20%3D%202383%5Cnvalue%20%3D%20%5B2181%2C%20202%5D%22%5D%20%3B%0A96%20-%3E%2097%20%3B%0A98%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.5%5Cngini%20%3D%200.05%5Cnsamples%20%3D%20546%5Cnvalue%20%3D%20%5B532%2C%2014%5D%22%5D%20%3B%0A97%20-%3E%2098%20%3B%0A99%20%5Blabel%3D%22gini%20%3D%200.039%5Cnsamples%20%3D%20506%5Cnvalue%20%3D%20%5B496%2C%2010%5D%22%5D%20%3B%0A98%20-%3E%2099%20%3B%0A100%20%5Blabel%3D%22gini%20%3D%200.18%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B36%2C%204%5D%22%5D%20%3B%0A98%20-%3E%20100%20%3B%0A101%20%5Blabel%3D%22hours-per-week%20%3C%3D%2059.5%5Cngini%20%3D%200.184%5Cnsamples%20%3D%201837%5Cnvalue%20%3D%20%5B1649%2C%20188%5D%22%5D%20%3B%0A97%20-%3E%20101%20%3B%0A102%20%5Blabel%3D%22gini%20%3D%200.18%5Cnsamples%20%3D%201789%5Cnvalue%20%3D%20%5B1610%2C%20179%5D%22%5D%20%3B%0A101%20-%3E%20102%20%3B%0A103%20%5Blabel%3D%22gini%20%3D%200.305%5Cnsamples%20%3D%2048%5Cnvalue%20%3D%20%5B39%2C%209%5D%22%5D%20%3B%0A101%20-%3E%20103%20%3B%0A104%20%5Blabel%3D%22age%20%3C%3D%2038.5%5Cngini%20%3D%200.067%5Cnsamples%20%3D%20718%5Cnvalue%20%3D%20%5B693%2C%2025%5D%22%5D%20%3B%0A96%20-%3E%20104%20%3B%0A105%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.0%5Cngini%20%3D%200.009%5Cnsamples%20%3D%20224%5Cnvalue%20%3D%20%5B223%2C%201%5D%22%5D%20%3B%0A104%20-%3E%20105%20%3B%0A106%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20213%5Cnvalue%20%3D%20%5B213%2C%200%5D%22%5D%20%3B%0A105%20-%3E%20106%20%3B%0A107%20%5Blabel%3D%22gini%20%3D%200.165%5Cnsamples%20%3D%2011%5Cnvalue%20%3D%20%5B10%2C%201%5D%22%5D%20%3B%0A105%20-%3E%20107%20%3B%0A108%20%5Blabel%3D%22hours-per-week%20%3C%3D%2036.5%5Cngini%20%3D%200.092%5Cnsamples%20%3D%20494%5Cnvalue%20%3D%20%5B470%2C%2024%5D%22%5D%20%3B%0A104%20-%3E%20108%20%3B%0A109%20%5Blabel%3D%22gini%20%3D%200.013%5Cnsamples%20%3D%20156%5Cnvalue%20%3D%20%5B155%2C%201%5D%22%5D%20%3B%0A108%20-%3E%20109%20%3B%0A110%20%5Blabel%3D%22gini%20%3D%200.127%5Cnsamples%20%3D%20338%5Cnvalue%20%3D%20%5B315%2C%2023%5D%22%5D%20%3B%0A108%20-%3E%20110%20%3B%0A111%20%5Blabel%3D%22hours-per-week%20%3C%3D%2044.5%5Cngini%20%3D%200.24%5Cnsamples%20%3D%202024%5Cnvalue%20%3D%20%5B1742%2C%20282%5D%22%5D%20%3B%0A95%20-%3E%20111%20%3B%0A112%20%5Blabel%3D%22race%20%3C%3D%200.5%5Cngini%20%3D%200.218%5Cnsamples%20%3D%201687%5Cnvalue%20%3D%20%5B1477%2C%20210%5D%22%5D%20%3B%0A111%20-%3E%20112%20%3B%0A113%20%5Blabel%3D%22age%20%3C%3D%2062.5%5Cngini%20%3D%200.239%5Cnsamples%20%3D%201309%5Cnvalue%20%3D%20%5B1127%2C%20182%5D%22%5D%20%3B%0A112%20-%3E%20113%20%3B%0A114%20%5Blabel%3D%22gini%20%3D%200.254%5Cnsamples%20%3D%201173%5Cnvalue%20%3D%20%5B998%2C%20175%5D%22%5D%20%3B%0A113%20-%3E%20114%20%3B%0A115%20%5Blabel%3D%22gini%20%3D%200.098%5Cnsamples%20%3D%20136%5Cnvalue%20%3D%20%5B129%2C%207%5D%22%5D%20%3B%0A113%20-%3E%20115%20%3B%0A116%20%5Blabel%3D%22hours-per-week%20%3C%3D%2037.5%5Cngini%20%3D%200.137%5Cnsamples%20%3D%20378%5Cnvalue%20%3D%20%5B350%2C%2028%5D%22%5D%20%3B%0A112%20-%3E%20116%20%3B%0A117%20%5Blabel%3D%22gini%20%3D%200.04%5Cnsamples%20%3D%2097%5Cnvalue%20%3D%20%5B95%2C%202%5D%22%5D%20%3B%0A116%20-%3E%20117%20%3B%0A118%20%5Blabel%3D%22gini%20%3D%200.168%5Cnsamples%20%3D%20281%5Cnvalue%20%3D%20%5B255%2C%2026%5D%22%5D%20%3B%0A116%20-%3E%20118%20%3B%0A119%20%5Blabel%3D%22age%20%3C%3D%2037.5%5Cngini%20%3D%200.336%5Cnsamples%20%3D%20337%5Cnvalue%20%3D%20%5B265%2C%2072%5D%22%5D%20%3B%0A111%20-%3E%20119%20%3B%0A120%20%5Blabel%3D%22educational-num%20%3C%3D%2010.5%5Cngini%20%3D%200.417%5Cnsamples%20%3D%2091%5Cnvalue%20%3D%20%5B64%2C%2027%5D%22%5D%20%3B%0A119%20-%3E%20120%20%3B%0A121%20%5Blabel%3D%22gini%20%3D%200.307%5Cnsamples%20%3D%2058%5Cnvalue%20%3D%20%5B47%2C%2011%5D%22%5D%20%3B%0A120%20-%3E%20121%20%3B%0A122%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B17%2C%2016%5D%22%5D%20%3B%0A120%20-%3E%20122%20%3B%0A123%20%5Blabel%3D%22age%20%3C%3D%2053.5%5Cngini%20%3D%200.299%5Cnsamples%20%3D%20246%5Cnvalue%20%3D%20%5B201%2C%2045%5D%22%5D%20%3B%0A119%20-%3E%20123%20%3B%0A124%20%5Blabel%3D%22gini%20%3D%200.261%5Cnsamples%20%3D%20194%5Cnvalue%20%3D%20%5B164%2C%2030%5D%22%5D%20%3B%0A123%20-%3E%20124%20%3B%0A125%20%5Blabel%3D%22gini%20%3D%200.411%5Cnsamples%20%3D%2052%5Cnvalue%20%3D%20%5B37%2C%2015%5D%22%5D%20%3B%0A123%20-%3E%20125%20%3B%0A126%20%5Blabel%3D%22age%20%3C%3D%2028.5%5Cngini%20%3D%200.499%5Cnsamples%20%3D%208476%5Cnvalue%20%3D%20%5B4413%2C%204063%5D%22%5D%20%3B%0A0%20-%3E%20126%20%5Blabeldistance%3D2.5%2C%20labelangle%3D-45%2C%20headlabel%3D%22False%22%5D%20%3B%0A127%20%5Blabel%3D%22age%20%3C%3D%2027.5%5Cngini%20%3D%200.197%5Cnsamples%20%3D%201473%5Cnvalue%20%3D%20%5B1310%2C%20163%5D%22%5D%20%3B%0A126%20-%3E%20127%20%3B%0A128%20%5Blabel%3D%22hours-per-week%20%3C%3D%2053.5%5Cngini%20%3D%200.143%5Cnsamples%20%3D%201242%5Cnvalue%20%3D%20%5B1146%2C%2096%5D%22%5D%20%3B%0A127%20-%3E%20128%20%3B%0A129%20%5Blabel%3D%22age%20%3C%3D%2024.5%5Cngini%20%3D%200.121%5Cnsamples%20%3D%201148%5Cnvalue%20%3D%20%5B1074%2C%2074%5D%22%5D%20%3B%0A128%20-%3E%20129%20%3B%0A130%20%5Blabel%3D%22hours-per-week%20%3C%3D%2034.0%5Cngini%20%3D%200.053%5Cnsamples%20%3D%20515%5Cnvalue%20%3D%20%5B501%2C%2014%5D%22%5D%20%3B%0A129%20-%3E%20130%20%3B%0A131%20%5Blabel%3D%22age%20%3C%3D%2023.5%5Cngini%20%3D%200.012%5Cnsamples%20%3D%20163%5Cnvalue%20%3D%20%5B162%2C%201%5D%22%5D%20%3B%0A130%20-%3E%20131%20%3B%0A132%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20111%5Cnvalue%20%3D%20%5B111%2C%200%5D%22%5D%20%3B%0A131%20-%3E%20132%20%3B%0A133%20%5Blabel%3D%22gini%20%3D%200.038%5Cnsamples%20%3D%2052%5Cnvalue%20%3D%20%5B51%2C%201%5D%22%5D%20%3B%0A131%20-%3E%20133%20%3B%0A134%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.071%5Cnsamples%20%3D%20352%5Cnvalue%20%3D%20%5B339%2C%2013%5D%22%5D%20%3B%0A130%20-%3E%20134%20%3B%0A135%20%5Blabel%3D%22gini%20%3D%200.063%5Cnsamples%20%3D%20340%5Cnvalue%20%3D%20%5B329%2C%2011%5D%22%5D%20%3B%0A134%20-%3E%20135%20%3B%0A136%20%5Blabel%3D%22gini%20%3D%200.278%5Cnsamples%20%3D%2012%5Cnvalue%20%3D%20%5B10%2C%202%5D%22%5D%20%3B%0A134%20-%3E%20136%20%3B%0A137%20%5Blabel%3D%22educational-num%20%3C%3D%2014.5%5Cngini%20%3D%200.172%5Cnsamples%20%3D%20633%5Cnvalue%20%3D%20%5B573%2C%2060%5D%22%5D%20%3B%0A129%20-%3E%20137%20%3B%0A138%20%5Blabel%3D%22hours-per-week%20%3C%3D%2037.5%5Cngini%20%3D%200.167%5Cnsamples%20%3D%20618%5Cnvalue%20%3D%20%5B561%2C%2057%5D%22%5D%20%3B%0A137%20-%3E%20138%20%3B%0A139%20%5Blabel%3D%22gini%20%3D%200.048%5Cnsamples%20%3D%20122%5Cnvalue%20%3D%20%5B119%2C%203%5D%22%5D%20%3B%0A138%20-%3E%20139%20%3B%0A140%20%5Blabel%3D%22gini%20%3D%200.194%5Cnsamples%20%3D%20496%5Cnvalue%20%3D%20%5B442%2C%2054%5D%22%5D%20%3B%0A138%20-%3E%20140%20%3B%0A141%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B12%2C%203%5D%22%5D%20%3B%0A137%20-%3E%20141%20%3B%0A142%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.359%5Cnsamples%20%3D%2094%5Cnvalue%20%3D%20%5B72%2C%2022%5D%22%5D%20%3B%0A128%20-%3E%20142%20%3B%0A143%20%5Blabel%3D%22hours-per-week%20%3C%3D%2071.0%5Cngini%20%3D%200.404%5Cnsamples%20%3D%2064%5Cnvalue%20%3D%20%5B46%2C%2018%5D%22%5D%20%3B%0A142%20-%3E%20143%20%3B%0A144%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.361%5Cnsamples%20%3D%2055%5Cnvalue%20%3D%20%5B42%2C%2013%5D%22%5D%20%3B%0A143%20-%3E%20144%20%3B%0A145%20%5Blabel%3D%22gini%20%3D%200.391%5Cnsamples%20%3D%2045%5Cnvalue%20%3D%20%5B33%2C%2012%5D%22%5D%20%3B%0A144%20-%3E%20145%20%3B%0A146%20%5Blabel%3D%22gini%20%3D%200.18%5Cnsamples%20%3D%2010%5Cnvalue%20%3D%20%5B9%2C%201%5D%22%5D%20%3B%0A144%20-%3E%20146%20%3B%0A147%20%5Blabel%3D%22gini%20%3D%200.494%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B4%2C%205%5D%22%5D%20%3B%0A143%20-%3E%20147%20%3B%0A148%20%5Blabel%3D%22gini%20%3D%200.231%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B26%2C%204%5D%22%5D%20%3B%0A142%20-%3E%20148%20%3B%0A149%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.412%5Cnsamples%20%3D%20231%5Cnvalue%20%3D%20%5B164%2C%2067%5D%22%5D%20%3B%0A127%20-%3E%20149%20%3B%0A150%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.447%5Cnsamples%20%3D%20148%5Cnvalue%20%3D%20%5B98%2C%2050%5D%22%5D%20%3B%0A149%20-%3E%20150%20%3B%0A151%20%5Blabel%3D%22hours-per-week%20%3C%3D%2056.0%5Cngini%20%3D%200.46%5Cnsamples%20%3D%20117%5Cnvalue%20%3D%20%5B75%2C%2042%5D%22%5D%20%3B%0A150%20-%3E%20151%20%3B%0A152%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.0%5Cngini%20%3D%200.438%5Cnsamples%20%3D%20108%5Cnvalue%20%3D%20%5B73%2C%2035%5D%22%5D%20%3B%0A151%20-%3E%20152%20%3B%0A153%20%5Blabel%3D%22gini%20%3D%200.413%5Cnsamples%20%3D%2072%5Cnvalue%20%3D%20%5B51%2C%2021%5D%22%5D%20%3B%0A152%20-%3E%20153%20%3B%0A154%20%5Blabel%3D%22gini%20%3D%200.475%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B22%2C%2014%5D%22%5D%20%3B%0A152%20-%3E%20154%20%3B%0A155%20%5Blabel%3D%22gini%20%3D%200.346%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B2%2C%207%5D%22%5D%20%3B%0A151%20-%3E%20155%20%3B%0A156%20%5Blabel%3D%22hours-per-week%20%3C%3D%2043.5%5Cngini%20%3D%200.383%5Cnsamples%20%3D%2031%5Cnvalue%20%3D%20%5B23%2C%208%5D%22%5D%20%3B%0A150%20-%3E%20156%20%3B%0A157%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B11%2C%202%5D%22%5D%20%3B%0A156%20-%3E%20157%20%3B%0A158%20%5Blabel%3D%22educational-num%20%3C%3D%2014.5%5Cngini%20%3D%200.444%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B12%2C%206%5D%22%5D%20%3B%0A156%20-%3E%20158%20%3B%0A159%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B6%2C%203%5D%22%5D%20%3B%0A158%20-%3E%20159%20%3B%0A160%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B6%2C%203%5D%22%5D%20%3B%0A158%20-%3E%20160%20%3B%0A161%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.326%5Cnsamples%20%3D%2083%5Cnvalue%20%3D%20%5B66%2C%2017%5D%22%5D%20%3B%0A149%20-%3E%20161%20%3B%0A162%20%5Blabel%3D%22hours-per-week%20%3C%3D%2044.0%5Cngini%20%3D%200.278%5Cnsamples%20%3D%2066%5Cnvalue%20%3D%20%5B55%2C%2011%5D%22%5D%20%3B%0A161%20-%3E%20162%20%3B%0A163%20%5Blabel%3D%22hours-per-week%20%3C%3D%2022.0%5Cngini%20%3D%200.194%5Cnsamples%20%3D%2046%5Cnvalue%20%3D%20%5B41%2C%205%5D%22%5D%20%3B%0A162%20-%3E%20163%20%3B%0A164%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B9%2C%200%5D%22%5D%20%3B%0A163%20-%3E%20164%20%3B%0A165%20%5Blabel%3D%22gini%20%3D%200.234%5Cnsamples%20%3D%2037%5Cnvalue%20%3D%20%5B32%2C%205%5D%22%5D%20%3B%0A163%20-%3E%20165%20%3B%0A166%20%5Blabel%3D%22gini%20%3D%200.42%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B14%2C%206%5D%22%5D%20%3B%0A162%20-%3E%20166%20%3B%0A167%20%5Blabel%3D%22gini%20%3D%200.457%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B11%2C%206%5D%22%5D%20%3B%0A161%20-%3E%20167%20%3B%0A168%20%5Blabel%3D%22gender%20%3C%3D%200.5%5Cngini%20%3D%200.494%5Cnsamples%20%3D%207003%5Cnvalue%20%3D%20%5B3103%2C%203900%5D%22%5D%20%3B%0A126%20-%3E%20168%20%3B%0A169%20%5Blabel%3D%22hours-per-week%20%3C%3D%2038.5%5Cngini%20%3D%200.465%5Cnsamples%20%3D%205221%5Cnvalue%20%3D%20%5B1916%2C%203305%5D%22%5D%20%3B%0A168%20-%3E%20169%20%3B%0A170%20%5Blabel%3D%22hours-per-week%20%3C%3D%2030.5%5Cngini%20%3D%200.476%5Cnsamples%20%3D%20586%5Cnvalue%20%3D%20%5B357%2C%20229%5D%22%5D%20%3B%0A169%20-%3E%20170%20%3B%0A171%20%5Blabel%3D%22age%20%3C%3D%2042.5%5Cngini%20%3D%200.428%5Cnsamples%20%3D%20342%5Cnvalue%20%3D%20%5B236%2C%20106%5D%22%5D%20%3B%0A170%20-%3E%20171%20%3B%0A172%20%5Blabel%3D%22hours-per-week%20%3C%3D%2026.0%5Cngini%20%3D%200.25%5Cnsamples%20%3D%2082%5Cnvalue%20%3D%20%5B70%2C%2012%5D%22%5D%20%3B%0A171%20-%3E%20172%20%3B%0A173%20%5Blabel%3D%22gini%20%3D%200.316%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B45%2C%2011%5D%22%5D%20%3B%0A172%20-%3E%20173%20%3B%0A174%20%5Blabel%3D%22gini%20%3D%200.074%5Cnsamples%20%3D%2026%5Cnvalue%20%3D%20%5B25%2C%201%5D%22%5D%20%3B%0A172%20-%3E%20174%20%3B%0A175%20%5Blabel%3D%22educational-num%20%3C%3D%2015.5%5Cngini%20%3D%200.462%5Cnsamples%20%3D%20260%5Cnvalue%20%3D%20%5B166%2C%2094%5D%22%5D%20%3B%0A171%20-%3E%20175%20%3B%0A176%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%20237%5Cnvalue%20%3D%20%5B158%2C%2079%5D%22%5D%20%3B%0A175%20-%3E%20176%20%3B%0A177%20%5Blabel%3D%22gini%20%3D%200.454%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B8%2C%2015%5D%22%5D%20%3B%0A175%20-%3E%20177%20%3B%0A178%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.5%5Cnsamples%20%3D%20244%5Cnvalue%20%3D%20%5B121%2C%20123%5D%22%5D%20%3B%0A170%20-%3E%20178%20%3B%0A179%20%5Blabel%3D%22race%20%3C%3D%200.5%5Cngini%20%3D%200.468%5Cnsamples%20%3D%20142%5Cnvalue%20%3D%20%5B89%2C%2053%5D%22%5D%20%3B%0A178%20-%3E%20179%20%3B%0A180%20%5Blabel%3D%22gini%20%3D%200.477%5Cnsamples%20%3D%20122%5Cnvalue%20%3D%20%5B74%2C%2048%5D%22%5D%20%3B%0A179%20-%3E%20180%20%3B%0A181%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B15%2C%205%5D%22%5D%20%3B%0A179%20-%3E%20181%20%3B%0A182%20%5Blabel%3D%22age%20%3C%3D%2041.5%5Cngini%20%3D%200.431%5Cnsamples%20%3D%20102%5Cnvalue%20%3D%20%5B32%2C%2070%5D%22%5D%20%3B%0A178%20-%3E%20182%20%3B%0A183%20%5Blabel%3D%22gini%20%3D%200.499%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B14%2C%2013%5D%22%5D%20%3B%0A182%20-%3E%20183%20%3B%0A184%20%5Blabel%3D%22gini%20%3D%200.365%5Cnsamples%20%3D%2075%5Cnvalue%20%3D%20%5B18%2C%2057%5D%22%5D%20%3B%0A182%20-%3E%20184%20%3B%0A185%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.446%5Cnsamples%20%3D%204635%5Cnvalue%20%3D%20%5B1559%2C%203076%5D%22%5D%20%3B%0A169%20-%3E%20185%20%3B%0A186%20%5Blabel%3D%22hours-per-week%20%3C%3D%2049.0%5Cngini%20%3D%200.477%5Cnsamples%20%3D%202855%5Cnvalue%20%3D%20%5B1124%2C%201731%5D%22%5D%20%3B%0A185%20-%3E%20186%20%3B%0A187%20%5Blabel%3D%22age%20%3C%3D%2036.5%5Cngini%20%3D%200.491%5Cnsamples%20%3D%201781%5Cnvalue%20%3D%20%5B770%2C%201011%5D%22%5D%20%3B%0A186%20-%3E%20187%20%3B%0A188%20%5Blabel%3D%22gini%20%3D%200.496%5Cnsamples%20%3D%20565%5Cnvalue%20%3D%20%5B309%2C%20256%5D%22%5D%20%3B%0A187%20-%3E%20188%20%3B%0A189%20%5Blabel%3D%22gini%20%3D%200.471%5Cnsamples%20%3D%201216%5Cnvalue%20%3D%20%5B461%2C%20755%5D%22%5D%20%3B%0A187%20-%3E%20189%20%3B%0A190%20%5Blabel%3D%22hours-per-week%20%3C%3D%2055.5%5Cngini%20%3D%200.442%5Cnsamples%20%3D%201074%5Cnvalue%20%3D%20%5B354%2C%20720%5D%22%5D%20%3B%0A186%20-%3E%20190%20%3B%0A191%20%5Blabel%3D%22gini%20%3D%200.413%5Cnsamples%20%3D%20694%5Cnvalue%20%3D%20%5B202%2C%20492%5D%22%5D%20%3B%0A190%20-%3E%20191%20%3B%0A192%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%20380%5Cnvalue%20%3D%20%5B152%2C%20228%5D%22%5D%20%3B%0A190%20-%3E%20192%20%3B%0A193%20%5Blabel%3D%22age%20%3C%3D%2032.5%5Cngini%20%3D%200.369%5Cnsamples%20%3D%201780%5Cnvalue%20%3D%20%5B435%2C%201345%5D%22%5D%20%3B%0A185%20-%3E%20193%20%3B%0A194%20%5Blabel%3D%22hours-per-week%20%3C%3D%2052.5%5Cngini%20%3D%200.494%5Cnsamples%20%3D%20152%5Cnvalue%20%3D%20%5B68%2C%2084%5D%22%5D%20%3B%0A193%20-%3E%20194%20%3B%0A195%20%5Blabel%3D%22gini%20%3D%200.499%5Cnsamples%20%3D%20106%5Cnvalue%20%3D%20%5B51%2C%2055%5D%22%5D%20%3B%0A194%20-%3E%20195%20%3B%0A196%20%5Blabel%3D%22gini%20%3D%200.466%5Cnsamples%20%3D%2046%5Cnvalue%20%3D%20%5B17%2C%2029%5D%22%5D%20%3B%0A194%20-%3E%20196%20%3B%0A197%20%5Blabel%3D%22educational-num%20%3C%3D%2014.5%5Cngini%20%3D%200.349%5Cnsamples%20%3D%201628%5Cnvalue%20%3D%20%5B367%2C%201261%5D%22%5D%20%3B%0A193%20-%3E%20197%20%3B%0A198%20%5Blabel%3D%22gini%20%3D%200.401%5Cnsamples%20%3D%20982%5Cnvalue%20%3D%20%5B272%2C%20710%5D%22%5D%20%3B%0A197%20-%3E%20198%20%3B%0A199%20%5Blabel%3D%22gini%20%3D%200.251%5Cnsamples%20%3D%20646%5Cnvalue%20%3D%20%5B95%2C%20551%5D%22%5D%20%3B%0A197%20-%3E%20199%20%3B%0A200%20%5Blabel%3D%22hours-per-week%20%3C%3D%2043.5%5Cngini%20%3D%200.445%5Cnsamples%20%3D%201782%5Cnvalue%20%3D%20%5B1187%2C%20595%5D%22%5D%20%3B%0A168%20-%3E%20200%20%3B%0A201%20%5Blabel%3D%22educational-num%20%3C%3D%2015.5%5Cngini%20%3D%200.409%5Cnsamples%20%3D%201244%5Cnvalue%20%3D%20%5B888%2C%20356%5D%22%5D%20%3B%0A200%20-%3E%20201%20%3B%0A202%20%5Blabel%3D%22educational-num%20%3C%3D%2013.5%5Cngini%20%3D%200.401%5Cnsamples%20%3D%201200%5Cnvalue%20%3D%20%5B867%2C%20333%5D%22%5D%20%3B%0A201%20-%3E%20202%20%3B%0A203%20%5Blabel%3D%22age%20%3C%3D%2058.5%5Cngini%20%3D%200.382%5Cnsamples%20%3D%20818%5Cnvalue%20%3D%20%5B608%2C%20210%5D%22%5D%20%3B%0A202%20-%3E%20203%20%3B%0A204%20%5Blabel%3D%22gini%20%3D%200.395%5Cnsamples%20%3D%20745%5Cnvalue%20%3D%20%5B543%2C%20202%5D%22%5D%20%3B%0A203%20-%3E%20204%20%3B%0A205%20%5Blabel%3D%22gini%20%3D%200.195%5Cnsamples%20%3D%2073%5Cnvalue%20%3D%20%5B65%2C%208%5D%22%5D%20%3B%0A203%20-%3E%20205%20%3B%0A206%20%5Blabel%3D%22age%20%3C%3D%2033.5%5Cngini%20%3D%200.437%5Cnsamples%20%3D%20382%5Cnvalue%20%3D%20%5B259%2C%20123%5D%22%5D%20%3B%0A202%20-%3E%20206%20%3B%0A207%20%5Blabel%3D%22gini%20%3D%200.33%5Cnsamples%20%3D%2048%5Cnvalue%20%3D%20%5B38%2C%2010%5D%22%5D%20%3B%0A206%20-%3E%20207%20%3B%0A208%20%5Blabel%3D%22gini%20%3D%200.448%5Cnsamples%20%3D%20334%5Cnvalue%20%3D%20%5B221%2C%20113%5D%22%5D%20%3B%0A206%20-%3E%20208%20%3B%0A209%20%5Blabel%3D%22age%20%3C%3D%2039.5%5Cngini%20%3D%200.499%5Cnsamples%20%3D%2044%5Cnvalue%20%3D%20%5B21%2C%2023%5D%22%5D%20%3B%0A201%20-%3E%20209%20%3B%0A210%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%209%5Cnvalue%20%3D%20%5B6%2C%203%5D%22%5D%20%3B%0A209%20-%3E%20210%20%3B%0A211%20%5Blabel%3D%22age%20%3C%3D%2046.0%5Cngini%20%3D%200.49%5Cnsamples%20%3D%2035%5Cnvalue%20%3D%20%5B15%2C%2020%5D%22%5D%20%3B%0A209%20-%3E%20211%20%3B%0A212%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B5%2C%2010%5D%22%5D%20%3B%0A211%20-%3E%20212%20%3B%0A213%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B10%2C%2010%5D%22%5D%20%3B%0A211%20-%3E%20213%20%3B%0A214%20%5Blabel%3D%22educational-num%20%3C%3D%2014.5%5Cngini%20%3D%200.494%5Cnsamples%20%3D%20538%5Cnvalue%20%3D%20%5B299%2C%20239%5D%22%5D%20%3B%0A200%20-%3E%20214%20%3B%0A215%20%5Blabel%3D%22hours-per-week%20%3C%3D%2062.5%5Cngini%20%3D%200.476%5Cnsamples%20%3D%20473%5Cnvalue%20%3D%20%5B288%2C%20185%5D%22%5D%20%3B%0A214%20-%3E%20215%20%3B%0A216%20%5Blabel%3D%22age%20%3C%3D%2058.5%5Cngini%20%3D%200.482%5Cnsamples%20%3D%20437%5Cnvalue%20%3D%20%5B260%2C%20177%5D%22%5D%20%3B%0A215%20-%3E%20216%20%3B%0A217%20%5Blabel%3D%22gini%20%3D%200.478%5Cnsamples%20%3D%20417%5Cnvalue%20%3D%20%5B252%2C%20165%5D%22%5D%20%3B%0A216%20-%3E%20217%20%3B%0A218%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B8%2C%2012%5D%22%5D%20%3B%0A216%20-%3E%20218%20%3B%0A219%20%5Blabel%3D%22age%20%3C%3D%2041.5%5Cngini%20%3D%200.346%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B28%2C%208%5D%22%5D%20%3B%0A215%20-%3E%20219%20%3B%0A220%20%5Blabel%3D%22gini%20%3D%200.401%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B13%2C%205%5D%22%5D%20%3B%0A219%20-%3E%20220%20%3B%0A221%20%5Blabel%3D%22gini%20%3D%200.278%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B15%2C%203%5D%22%5D%20%3B%0A219%20-%3E%20221%20%3B%0A222%20%5Blabel%3D%22hours-per-week%20%3C%3D%2062.5%5Cngini%20%3D%200.281%5Cnsamples%20%3D%2065%5Cnvalue%20%3D%20%5B11%2C%2054%5D%22%5D%20%3B%0A214%20-%3E%20222%20%3B%0A223%20%5Blabel%3D%22hours-per-week%20%3C%3D%2045.5%5Cngini%20%3D%200.198%5Cnsamples%20%3D%2054%5Cnvalue%20%3D%20%5B6%2C%2048%5D%22%5D%20%3B%0A222%20-%3E%20223%20%3B%0A224%20%5Blabel%3D%22gini%20%3D%200.42%5Cnsamples%20%3D%2010%5Cnvalue%20%3D%20%5B3%2C%207%5D%22%5D%20%3B%0A223%20-%3E%20224%20%3B%0A225%20%5Blabel%3D%22gini%20%3D%200.127%5Cnsamples%20%3D%2044%5Cnvalue%20%3D%20%5B3%2C%2041%5D%22%5D%20%3B%0A223%20-%3E%20225%20%3B%0A226%20%5Blabel%3D%22gini%20%3D%200.496%5Cnsamples%20%3D%2011%5Cnvalue%20%3D%20%5B5%2C%206%5D%22%5D%20%3B%0A222%20-%3E%20226%20%3B%0A%7D%0A)

In [ ]:
#Оценка важности признаков
optimal_tree.feature_importances_

# 3 Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier(n_estimators = 100, random_state = 8, max_depth = 6) 
rforest.fit(X_train, y_train)
y_pred = rforest.predict(X_valid)

accuracy_score(y_valid, y_pred)

In [ ]:
#Подбор гиперпараметров для случайного леса
forest_params = {'n_estimators': np.arange(1,5),'max_depth': np.arange(2, 5), 'min_samples_split': np.arange(2, 5), 
               'min_samples_leaf': np.arange(1,5), 'max_features': np.arange(1, 5)}

forest_grid = GridSearchCV(rforest, forest_params, cv=5, scoring='accuracy')
forest_grid.fit(X_train, y_train)

In [ ]:
forest_grid.best_params_

In [ ]:
rforest = RandomForestClassifier(n_estimators = 4, max_features = 3, min_samples_leaf = 1, min_samples_split = 2,
                                 random_state = 8, max_depth = 4) 
rforest.fit(X_train, y_train)
y_pred = rforest.predict(X_valid)

accuracy_score(y_valid, y_pred)

In [ ]:
rforest.feature_importances_

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

d = {'column': list(X.keys()), 'importances': list(rforest.feature_importances_)}
ax = sns.barplot(x = 'column', y = 'importances', data = d)

plt.title('Feature Importances')
ax.set(xlabel='Feature', ylabel='Importances')